# NOTES

1/2 BS

INFERENCE
- Pipeline
    - History: Keep last 100 interactions per user in memory
    - Build test sequence on-fly from current test_df and history for each user
    - Avoid workers > 0 in Pytorch dataloader, batches are small due to API
    - safeguard against a batch that just has a sinle lecture in it with `
    score = model.predict(inputs) if X_tes.shape[0] > 0 else np.array([], dtype = np.float32)`
    - build pipeline with all train data, not just the data we trained on!!!
- power siamese network ensembling

LAST DITCH
- NoamOpt
    - https://lab-ml.com/labml_nn/optimizers/noam.html
    - https://www.kaggle.com/adityaecdrid/fork-of-saint-inference-ea970c/comments
- collapse infrequent content_id embedding
- verify prior_question_had_explanation has signal or does it just cause overfit??
- float32 to speed things up?
- 1/2 BS
- drop all continuous features into a single subnet before feeding to rnn
- ++emb/++rnn dropout
- pretrain: ?
- userid model when we have 10 or more interactions :-) and non-userid model
- ffn encoder layer
- content id mean ordering vs user ordering difference

```
explanation_agg = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['sum', 'count'])
explanation_agg=explanation_agg.astype('int16')
cum = train_df.groupby('user_id')['prior_question_had_explanation'].agg(['cumsum', 'cumcount'])
cum['cumcount']=cum['cumcount']+1
train_df['explanation_mean'] = cum['cumsum'] / cum['cumcount']
```

**NOTES**
- Host's papers (https://arxiv.org/abs/2002.07033, https://arxiv.org/abs/2010.12042).

# Reports

## Findings v2

- [conjecture] we believe recent correct/incorrect streak has benefit
- [conjecture] all time correct/incorrect streaks bias the user too much?
    - recommend dropping same response streak
    - recommend dropping alltime correct/incorrect streaks
    - short term streak also absorbs the last 3,5,7 TCID mean feature
- [fail] try adding encoder into decoder embedding as well..
- [no change] no attention encoder, e.g. just a FFN/MLP and heavier decoder - surprisingly, this doesnt actually reduce the model's performance but 0.0001 if even.
- [useless] content id mean ordering vs user ordering
- [useless] user_answered longest same value streak - looks like reduces overfitting a bit :-)
    - seems to be a strong regularizer
    - stops our overfitting but doesn't increase score
    - the hit we take is 0.00025
    - but maybe too strong
- [good] post training user embedding fine tuning provides 0.0001 boost =E
- [meh] pretrain all network except exerciseid against target, then fine tune exercise id
- [meh] content_id rank diff causes slow overfit
- [superbad] SQRT(LEN//2) for TCID Shuffling
- [horrible] linear capped times rather than log scaled
- [good] slight tcid shuffling and no preferential sampling for tail of long users
- [no change] experience = cumtime spent answering, log encoded
- [no change] cumulative time spent answering for the session
- [no change] log(ms) instead of log(s)
- [bad] history moved to the embedder side...
- [bad] history merged into MEMORY (after embedder) going into decoder
- [okay] target clustered diagnostic features barely move the envelope
- [no diff] 2,3, vs 4,6   vs 6,9
- [bad] target mask off training
- [okay] other folds have similar scores....
- [good] inplace intra tcid shuffling augmentation works without issue, slight boost
-- [**great**] maasking and nan removal of history (correctness)
- [nogood] testing 128 w/o dropout; but train score caps out at 7990 ish.....
- [bad] tsvd/pca/clustering on tag_group_id
- [bad] all forms of difficulty
- BNorm [bad] on Linear inputs w/o Bias
- [bad] 256 w 4layer 2epoch warmup
- [nochange] dim_feedforward=d_model x 2 rather than just d_model
- [nochange] xent instead of bce
Tested
- prior_question_elapsed_time_embeddings shifted decoder
- prior_question_had_explanation_embeddings shifted decoder
- pos_embedding shifted decoder
- user_correctness patch for lectures
- ts embeddings shifted decoder <-- this hurt score a lot so rejected
- filtering <5 TCID users, and do trimming before processing
- attention masks: https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/206620
- why is difficulty shifted???????
    - dunno, but it was giving us better score
    - currently not shifted
- correctness_u_recency and difficulty are on different scales than the rest of the features flowing into the layer
- perhaps a layer norm first..

### Findings

- [**great**] Smaller BS (64)—maybe even no dropout. But this makes iterating slow... maybe dual gpu...
- [**great**] stock transformerencoder/decoder layers are better
- [**great**] larger ffn dim, at least d_model...
- [**great**] sin pos embedder
- [**great**] bundle id embedding: bundle groups the questions. task_container_id is a subset of q's in a bundle which are selected to be asked.
- [**great**] diff1 and diff2 interaction ts, no differentiation between Q & L
- [**great**] 20% encoder dropout. we need to test further increases
- [**great**] previous question correctness embedding
- [**great**] content type id embedding
- [**great**] data parallel
- [**great**] patched float64 stdscaler overflow issues
- [**great**] everything on the encoder except correctness and pos emb - still using IRT
- [**great**] no decoder needed it seems
- [**great**] larger CAP_SAMPLING_LENGTH_TRN seems to help, which makes sense since those longer users will be part of subset. test against lb though to optimize for this
- [**great**] user correctness mean
- [**great**] fp16 double speed
- [**great**] pre-sorting parquets to save time...
- [**great**] CV from: `kaggle kernels output marisakamozz/cv-strategy-in-the-kaggle-environment`
---
- [good] x2 = 8 layer stack more important than x2 = 512 dimension model...
- [good] important to seed numpy/random in different dataset loader threads
- [good] swapping memory+target on the decoder is competitive and might make for good ensemble
- [good] static pos embedding
- [good] train sampling: put more weight on further sequence to balance start sequences
- [good] big model dim, 256 or 512 even
- [good] question features on encoder, response features on decoder
- [good] heavier tail is marginally better... have to use dropout there though
- [good] updated to unbiased CV: https://www.kaggle.com/marisakamozz/cv-strategy-in-the-kaggle-environment
- [good] start_offset prefers later user interaction sequences
- [good] incorrect rank
- [good] label smoothing - cuts down speeds in 1/2 though....
- [good] bxe w/ dense targets
- [good] masked lectures...
- [good] fixed bug in embedding incorrect_rank (+3)
- [good] padding_idx=0 for all embeddings
- [good] tag community network clustering
- [good] actually, looks like relus beat tanh/gelu
---
- [meh] timestamp_u_incorrect_recency, timestamp_u_correct_recency features. I'm keeping them for now but they literally did nothing for AUC but may make us start to overfit.
- [meh] task_container_id doesn't really seem to move the needle; neither embedding nor log transform (NEEDS RETESTING)
- [meh] weight decay slows down convergence by 25% but seems to help sliiiiightly marginally? disabling for now we can re-enable in the future
- [meh] filling padded targets with mean; pretty sure this does nothing
- [meh] adding ALL tag embeddings (eats a lot of memory too) and the advantage is epsilon
- [meh] changing dropout {0, 0.1, 0.2} doesn't seem to make a difference..
- [meh] consolidation of continuous features into a single dense layer expended to d_model w/ tanh activation. Results actually degraded slightly; but this gives us the ability to further process through, e.g. BatchNorm, dropout, linear, before feeding into the Transformer.
- [meh] incorrect rank is now the target, xent loss, and weight tieing. Results aren't bad except with weight typing, they drop 0.0001. But without tieing, might be good for ensemble..
- [meh] adamw and adam preform similarly. asgd/sgd w/ large momentum, converge too slowly but converg'ish...
- [meh] item response theory output (alpha and beta) didn't really move the needle vs sigmoid
- [meh] ffn on exercise embedding / full src embedding before feeding into stack, marginal changes
- [meh] user_part_cummeans no real impact
- [meh] cumulative mean based on adjusted correctness (by question difficulty) 
---
- [bad] removing bias from linear embedder layers
- [bad] ts and its transforms no predictive power. HOWEVER, we might be able to break into two groups... explore..
- [bad] first bundle id no predictive power
- [bad] ts offset had no predictive power
- [bad] tags have no predictive power...
- [bad] weight decay, even only on non bias/norm layers...
- [bad] num_workers>0 in dataloader forces a seed, which isn't desired since we have random sampling augmentation. also inference batches are small so >n workers slows things down due to overhead.
- [bad] padding in collater has diminishing returns due to sampling preferring full length batches; not even useful in inference, since we cannot arrange users by length due to API ordering
- [bad] move continuous features to the decoder
- [bed] dense: BN,LinearNoBias,ReLU,Dropout,Linear,TanH
- [bad] swap memory + input for decoder
- [bad] container id embeddings
- [bad] IRT correct for `no_responses`
- [bad] user_lecture_lv: which is cumsum lectures / content
- [bad] user_lecture_ts: timestamp / cumcount(content)
- [bad] ts_since_lecture doesnt move the needle
- [bad] mean difficulty, even lagged.
- [bad] community and tag do nothing...

### Sampling Strategy

- Take all user ids in training data.
- Compute their lengths and cap the result to 500.
- Scale the prior to make them sum 1 (probability distribution).
- Sample N ids with replacement with previous computed probabilities. N in my case is the same as different user ids you have in the training data. The result will have repeated ids.
- Take a random sequence for every id. It may lead to repeated sequences with low probability. For example, if you have id 115 repeated 7 times, you will take 7 random sequences for the user 115.
- Repeat each epoch.

# Setup

In [ ]:
PRELOAD = False

import numpy as np
import pandas as pd
import gc, joblib, random

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pathlib import Path
from typing import List
from collections import defaultdict#,deque, namedtuple, Counter
from bitarray import bitarray

from time import time

# import itertools
# from contextlib import contextmanager

import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy

from einops import repeat, rearrange
from sklearn.metrics import accuracy_score, roc_auc_score
from prettytable import PrettyTable

TRAIN_DTYPES = {
    # 'row_id': np.uint32,
    'timestamp': np.uint64,
    'user_id': np.uint32,
    'content_id': np.uint16,
    'content_type_id': np.uint8,
    'task_container_id': np.uint16,
    'user_answer': np.int8,
    'answered_correctly': np.int8,
    'prior_question_elapsed_time': np.float32,
    'prior_question_had_explanation': 'boolean'
}

DATA_DIR = Path('../input/riiid-test-answer-prediction')
TRAIN_PATH = DATA_DIR / 'train.csv'
QUESTIONS_PATH = DATA_DIR / 'questions.csv'
LECTURES_PATH = DATA_DIR / 'lectures.csv'

In [ ]:
DEVICE = 'cuda'
N_EPOCHS = 50
BATCH_SIZE = 512 #// 2

# Must be >= window_size
CAP_SAMPLING_LENGTH_TRN = 2000
CAP_SAMPLING_LENGTH_VAL = 500

# this parameter denotes how many last seen content_ids I am going to consider
# <aka the max_seq_len or the window size>.
WINDOW_SIZE = 128

PAD = 0

GREAT_SEED = 1337

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = False
    
seed_everything(GREAT_SEED)
gc.enable()

In [ ]:
# Start col id labeling from this row below
COL_OFFSET_CUT = 1

dataset_cols = [
    # Leave user_id first, so we can truncate it
    'timestamp',
    'user_id',
    'task_container_id',
    
    #############
    # We start indexing from here, since the rest get cut off:
    'content_id',
    'part_id',
    'prior_question_elapsed_time',
    'prior_question_had_explanation',
    'incorrect_rank', # decoder feature that needs help..........
    'content_type_id',
    'bundle_id',
    
    # only these of the 4 have signal (0-index)
    'answer_ratio1',
    'answer_ratio2',
    'correct_streak_u',
    'incorrect_streak_u',
    'correct_streak_alltime_u',
    'incorrect_streak_alltime_u',
    
    # our session information + a lifetime field (elapsed time sum on platform—probably should be capped)
    'session_content_num_u',
    'session_duration_u',
    'session_ans_duration_u',
    'lifetime_ans_duration_u',
    
    # First block of continuous - timestamp intervals
    'lag_ts_u_recency',
    'last_ts_u_recency1',
    'last_ts_u_recency2',
    'last_ts_u_recency3',
    'last_correct_ts_u_recency',
    'last_incorrect_ts_u_recency',
    
    # Second block of continuous - how often user is right on global/part/session basis
    'correctness_u_recency',
    'part_correctness_u_recency',
    'session_correctness_u',
    
    # Have we been served this content before?
    'encountered',
    
    # Average score of most frequent asked questions
    'diagnostic_u_recency_1',
    'diagnostic_u_recency_2',
    'diagnostic_u_recency_3',
    'diagnostic_u_recency_4',
    'diagnostic_u_recency_5',
    'diagnostic_u_recency_6',

    'answered_correctly',
]

COL_USER_ID = dataset_cols.index('user_id') - COL_OFFSET_CUT
COL_TASK_CONTAINER_ID = dataset_cols.index('task_container_id') - COL_OFFSET_CUT
COL_CONTENT_ID = dataset_cols.index('content_id') - COL_OFFSET_CUT
COL_PART_ID = dataset_cols.index('part_id') - COL_OFFSET_CUT
COL_PRIOR_QUESTION_ELAPSED_TIME = dataset_cols.index('prior_question_elapsed_time') - COL_OFFSET_CUT
COL_PRIOR_QUESTION_EXPLANATION = dataset_cols.index('prior_question_had_explanation') - COL_OFFSET_CUT
COL_INCORRECT_RANK = dataset_cols.index('incorrect_rank') - COL_OFFSET_CUT
COL_CONTENT_TYPE_ID = dataset_cols.index('content_type_id') - COL_OFFSET_CUT
COL_BUNDLE_ID = dataset_cols.index('bundle_id') - COL_OFFSET_CUT

COL_ANSWER_RATIO1 = dataset_cols.index('answer_ratio1') - COL_OFFSET_CUT
COL_ANSWER_RATIO2 = dataset_cols.index('answer_ratio2') - COL_OFFSET_CUT
COL_CORRECT_STREAK_U = dataset_cols.index('correct_streak_u') - COL_OFFSET_CUT
COL_INCORRECT_STREAK_U = dataset_cols.index('incorrect_streak_u') - COL_OFFSET_CUT
COL_CORRECT_STREAK_ALLTIME_U = dataset_cols.index('correct_streak_alltime_u') - COL_OFFSET_CUT
COL_INCORRECT_STREAK_ALLTIME_U = dataset_cols.index('incorrect_streak_alltime_u') - COL_OFFSET_CUT
    
COL_SESSION_CONTENT_NUM_U = dataset_cols.index('session_content_num_u') - COL_OFFSET_CUT
COL_SESSION_DURATION_U = dataset_cols.index('session_duration_u') - COL_OFFSET_CUT
COL_SESSION_CORRECTNESS_U = dataset_cols.index('session_correctness_u') - COL_OFFSET_CUT
COL_SESSION_ANS_DURATION_U = dataset_cols.index('session_ans_duration_u') - COL_OFFSET_CUT
COL_LIFETIME_ANS_DURATION_U = dataset_cols.index('lifetime_ans_duration_u') - COL_OFFSET_CUT

COL_LAG_TS_RECENCY = dataset_cols.index('lag_ts_u_recency') - COL_OFFSET_CUT
COL_LAST_TS_RECENCY1 = dataset_cols.index('last_ts_u_recency1') - COL_OFFSET_CUT
COL_LAST_TS_RECENCY2 = dataset_cols.index('last_ts_u_recency2') - COL_OFFSET_CUT
COL_LAST_TS_RECENCY3 = dataset_cols.index('last_ts_u_recency3') - COL_OFFSET_CUT
COL_CORRECTNESS_U_RECENCY = dataset_cols.index('correctness_u_recency') - COL_OFFSET_CUT
COL_PART_CORRECTNESS_U_RECENCY = dataset_cols.index('part_correctness_u_recency') - COL_OFFSET_CUT
COL_LAST_CORRECT_TS_U_RECENCY = dataset_cols.index('last_correct_ts_u_recency') - COL_OFFSET_CUT
COL_LAST_INCORRECT_TS_U_RECENCY = dataset_cols.index('last_incorrect_ts_u_recency') - COL_OFFSET_CUT

COL_ENCOUNTERED = dataset_cols.index('encountered') - COL_OFFSET_CUT

COL_DIAGNOSTIC_RECENCY_1 = dataset_cols.index('diagnostic_u_recency_1') - COL_OFFSET_CUT
COL_DIAGNOSTIC_RECENCY_2 = dataset_cols.index('diagnostic_u_recency_2') - COL_OFFSET_CUT
COL_DIAGNOSTIC_RECENCY_3 = dataset_cols.index('diagnostic_u_recency_3') - COL_OFFSET_CUT
COL_DIAGNOSTIC_RECENCY_4 = dataset_cols.index('diagnostic_u_recency_4') - COL_OFFSET_CUT
COL_DIAGNOSTIC_RECENCY_5 = dataset_cols.index('diagnostic_u_recency_5') - COL_OFFSET_CUT
COL_DIAGNOSTIC_RECENCY_6 = dataset_cols.index('diagnostic_u_recency_6') - COL_OFFSET_CUT

COL_ANSWERED_CORRECTLY = dataset_cols.index('answered_correctly') - COL_OFFSET_CUT


In [ ]:
class UserDF():
    def pop_if_needed(self):
        # Ensure we're only keeping the final 100:
        if self.lensize[0] < 128:
            return True
        
        self.user_id.pop(0)
        self.task_container_id.pop(0)

        #############
        # We start indexing from here, since the rest get cut off:
        self.content_id.pop(0)
        self.part_id.pop(0)
        self.prior_question_elapsed_time.pop(0)
        self.prior_question_had_explanation.pop(0)
        self.incorrect_rank.pop(0) # decoder feature that needs help..........
        self.content_type_id.pop(0)
        self.bundle_id.pop(0)

        # only these of the 4 have signal (0-index)
        self.answer_ratio1.pop(0)
        self.answer_ratio2.pop(0)
        self.correct_streak_u.pop(0)
        self.incorrect_streak_u.pop(0)
        self.correct_streak_alltime_u.pop(0)
        self.incorrect_streak_alltime_u.pop(0)

        # our session information + a lifetime field (elapsed time sum on platform—probably should be capped)
        self.session_content_num_u.pop(0)
        self.session_duration_u.pop(0)
        self.session_ans_duration_u.pop(0)
        self.lifetime_ans_duration_u.pop(0)

        # First block of continuous - timestamp intervals
        self.lag_ts_u_recency.pop(0)
        self.last_ts_u_recency1.pop(0)
        self.last_ts_u_recency2.pop(0)
        self.last_ts_u_recency3.pop(0)
        self.last_correct_ts_u_recency.pop(0)
        self.last_incorrect_ts_u_recency.pop(0)

        # Second block of continuous - how often user is right on global/part/session basis
        self.correctness_u_recency.pop(0)
        self.part_correctness_u_recency.pop(0)
        self.session_correctness_u.pop(0)

        # Have we been served this content before?
        self.encountered.pop(0)

        # Average score of most frequent asked questions
        self.diagnostic_u_recency_1.pop(0)
        self.diagnostic_u_recency_2.pop(0)
        self.diagnostic_u_recency_3.pop(0)
        self.diagnostic_u_recency_4.pop(0)
        self.diagnostic_u_recency_5.pop(0)
        self.diagnostic_u_recency_6.pop(0)

        self.answered_correctly.pop(0)
        return False
            
    def __init__(self):
        # TODO: We may need to add some variables to keep track of how many predictions
        # And where in test_df they are
        self.lensize = [0]
        
        ##########
        self.user_id = []
        self.task_container_id = []

        #############
        # We start indexing from here, since the rest get cut off:
        self.content_id = []
        self.part_id = []
        self.prior_question_elapsed_time = []
        self.prior_question_had_explanation = []
        self.incorrect_rank = [] # decoder feature that needs help..........
        self.content_type_id = []
        self.bundle_id = []

        # only these of the 4 have signal (0-index)
        self.answer_ratio1 = []
        self.answer_ratio2 = []
        self.correct_streak_u = []
        self.incorrect_streak_u = []
        self.correct_streak_alltime_u = []
        self.incorrect_streak_alltime_u = []

        # our session information + a lifetime field (elapsed time sum on platform—probably should be capped)
        self.session_content_num_u = []
        self.session_duration_u = []
        self.session_ans_duration_u = []
        self.lifetime_ans_duration_u = []

        # First block of continuous - timestamp intervals
        self.lag_ts_u_recency = []
        self.last_ts_u_recency1 = []
        self.last_ts_u_recency2 = []
        self.last_ts_u_recency3 = []
        self.last_correct_ts_u_recency = []
        self.last_incorrect_ts_u_recency = []

        # Second block of continuous - how often user is right on global/part/session basis
        self.correctness_u_recency = []
        self.part_correctness_u_recency = []
        self.session_correctness_u = []

        # Have we been served this content before?
        self.encountered = []

        # Average score of most frequent asked questions
        self.diagnostic_u_recency_1 = []
        self.diagnostic_u_recency_2 = []
        self.diagnostic_u_recency_3 = []
        self.diagnostic_u_recency_4 = []
        self.diagnostic_u_recency_5 = []
        self.diagnostic_u_recency_6 = []

        self.answered_correctly = []

In [ ]:
class UserFeatures():
    # lagtime = previous bundle's tsdiff - previousquestionelapsedtime * previousbundlesize
    # account for lectures.
    
    def __init__(self):
        # Everything needs to be a list so we can pass references..
        # Idx=0 current ts tcid, Idx=1 start idx of this TCID, Idx=2 counter of items in TCID
        # Idx=0 current ts tcid, Idx=1 counter, Idx=2 is [idx,idx,idx]
        self.cached_task_container_id_u = [np.nan]

        # These two features are only used to compute the lag feature.
        # Index: 0 = current, 1=previous
        self.task_container_num_questions_u = [0,0]  # Only updated on question_id's!
        # Index: 0 = ts, 1=per-bundle1, 1 = per-bundle2,
        self.last_question_ts_u_recency2 = [np.nan,np.nan,np.nan]

        # Idx0=ts, Idx1=CorrectTS, Idx2=CountTS, Idx3=CorrectBundleNoLeak, Idx4=CountBundleBundleNoLeak
        # Idx5=CumAnsTimeSessionNoLeak, Idx6=CumAnsTimePlatformNoLeak
        self.session_u = [np.nan, 2,3, 2,3, 0,0]

        # Index: 0=ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3, ...
        self.last_ts_u = [np.nan,np.nan,np.nan,np.nan]

        # Index: 0 = per-ts, 1 = per-bundle
        self.last_correct_ts_u = [0,0] #[np.nan,np.nan]

        # Index: 0,1 = per-ts, 2,3 = per-bundle. 0=correct qs, 1=total qs
        self.last_incorrect_ts_u = [0,0] #[np.nan,np.nan]

        # Index: 0,1 = per-ts, 2,3 = per-bundle. 0=correct qs, 1=total qs
        self.correctness_u = [2,3,2,3]  # 4right,2wrong=66.7%, mean target

        # 7*4, [7parts][0=correct_ts,1=count_ts,2=correct_bundle,3=count_bundle]
        self.part_correctness_u = 7 * [[2,3,2,3]]

        # diagnostic_content_id_order_map
        # diagnostic_content_id_mean_map
        # diagnostic_content_id_mean_map_values
        self.diagnostic_u_1 = [
            # Target Grouping rather than association grouping for now:
            #26 	6911 	0.818782
            #33 	7901 	0.824779

            # First two indices are 'update indicators', last two are default values that are set at end of each TCID
            np.nan, np.nan, 0.818782, 0.824779,

            # Last value is default mean
            0.8217805
        ]

        self.diagnostic_u_2 = [
            # 30 	7219 	0.594133
            # 25 	6910 	0.602537
            # 4 	2066 	0.614043
            # 21 	6879 	0.619330
            # 3 	2065 	0.621588
            # 1 	1279 	0.635821
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
            0.594133, 0.602537, 0.614043, 0.619330, 0.621588, 0.635821,

            0.6145753333333334
        ]

        self.diagnostic_u_3 = [
            # 12 	3365 	0.532004
            # 16 	4697 	0.545093
            # 5 	2594 	0.557091
            np.nan, np.nan, np.nan,
            0.532004, 0.545093, 0.557091,

            0.5447293333333334
        ]

        self.diagnostic_u_4 = [
            # 20 	6878 	0.430956
            # 27 	6912 	0.431397
            # 15 	4493 	0.440798
            # 7 	2596 	0.468018
            # 29 	7218 	0.485773
            np.nan, np.nan, np.nan, np.nan, np.nan,
            0.430956, 0.431397, 0.440798, 0.468018, 0.485773,

            0.4513884
        ]

        self.diagnostic_u_5 = [
            # 24 	6909 	0.390869
            # 32 	7877 	0.393220
            # 22 	6880 	0.404521
            # 28 	7217 	0.405293
            # 6 	2595 	0.419734
            np.nan, np.nan, np.nan, np.nan, np.nan,
            0.390869, 0.393220, 0.404521, 0.405293, 0.419734,
            0.4027274
        ]

        self.diagnostic_u_6 = [
            # 10 	2949 	0.211211
            # 9 	2948 	0.226543
            # 14 	4121 	0.230986
            # 23 	6881 	0.252001
            # 18 	6174 	0.255422
            # 31 	7220 	0.262637
            # 11 	3364 	0.268132
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
            0.211211, 0.226543, 0.230986, 0.252001, 0.255422, 0.262637, 0.268132,
            0.24384742857142858,
        ]
        
        #user_answer_streak_map
        self.answer_ratio = [
            # Idx0 = last user answer, Idx1 = streak count
            # We don't have to limit to bundles since we have access to this at read time
            1,1,  # UA=1 counter, UA=1 bundle 
            1,1,  # UA=2 counter, UA=2 bundle
            4,4,  # total q asked
        ]
        
        self.streak_u = [
            0,0,0, # correct ts, bundle, all-time bundle
            0,0,0, # incorrect ts, bundle, all-time bundle
        ]

        # We don't need to update @ Bundle because we'll only see a question once in a TCID
        self.content_encounter = bitarray(13550+425, endian='little')
        self.content_encounter.setall(0)
        
    def get_features(self):
        return (
            self.cached_task_container_id_u,
            self.task_container_num_questions_u,
            self.last_question_ts_u_recency2,
            self.session_u,
            self.last_ts_u,
            self.last_correct_ts_u,
            self.last_incorrect_ts_u,
            self.correctness_u,
            self.part_correctness_u,
            self.diagnostic_u_1,
            self.diagnostic_u_2,
            self.diagnostic_u_3,
            self.diagnostic_u_4,
            self.diagnostic_u_5,
            self.diagnostic_u_6,
            self.answer_ratio,
            self.streak_u,
            self.content_encounter,
        )

In [ ]:
# For pickling
def newUserFeatures(): return UserFeatures()
def newUserDF(): return UserDF()

In [ ]:
def prepare_training_data(fold_number=0):
    t = time()
    
    # +1 for padding idx
    df_lectures = pd.read_csv(LECTURES_PATH)
    df_questions = pd.read_csv(QUESTIONS_PATH)
    #df_questions_lsi = pd.read_csv('./genism_question_lsi20.csv')[['content_id', 'tags_lsi']]

    # Since we increment content_id...
    #df_questions_lsi.content_id += 1
    df_questions.question_id += 1
    df_lectures.lecture_id += 1
    
    mqid = df_questions.question_id.max() + 1
    lecture_ids_map = {v: i+mqid for i,v in enumerate(df_lectures.lecture_id.unique()) }
    df_lectures.lecture_id = df_lectures.lecture_id.map(lecture_ids_map)
    

    # Parts are: [1, 2, 3, 4, 5, 6, 7]
    # So 0 already reserved (and not mapped); 7+1 total
    # No join needed:
    # NOTE: This is mapping only the first part for questions!!!!!!!!!!!!!!!!!!!!
    part_ids_map = dict(zip(df_questions.question_id, df_questions.part))
    part_ids_map.update(dict(zip(df_lectures.lecture_id, df_lectures.part)))

#     df_train = pd.read_parquet(f'../input/uthman-cv/cv{fold_number+1}_train.parquet')
#     df_valid = pd.read_parquet(f'../input/uthman-cv/cv{fold_number+1}_valid.parquet')
    
    df_train = pd.read_parquet(f'../input/cv-strategy-in-the-kaggle-environment/cv{fold_number+1}_train_sorted.parquet')[TRAIN_DTYPES.keys()]
    df_valid = pd.read_parquet(f'../input/cv-strategy-in-the-kaggle-environment/cv{fold_number+1}_valid_sorted.parquet')[TRAIN_DTYPES.keys()]    

#     df_train = pd.read_pickle(f'../input/riiid-cross-validation-files/cv{fold_number+1}_train.pickle')
#     df_valid = pd.read_pickle(f'../input/riiid-cross-validation-files/cv{fold_number+1}_valid.pickle')
#     df_train.sort_values(['user_id','timestamp'], inplace=True)
#     df_valid.sort_values(['user_id','timestamp'], inplace=True)
    
# #     # NOTE: REMOVE:
#     df_train = df_train.iloc[:5000000]
#     df_valid = df_valid.iloc[:5000000]
    
    #Filter
    df_train = df_train.groupby('user_id', sort=False).head(CAP_SAMPLING_LENGTH_TRN)
    df_valid = df_valid.groupby('user_id', sort=False).head(CAP_SAMPLING_LENGTH_VAL)
    # NOTE: User IDs already mapped.... but we add 1 so that we can use 0 for unknown users
    embedded_user_ids_map = joblib.load('../input/uthman-cv/mapped_embedded_user_ids.jlib')
    embedded_user_ids_map = {i:v+1 for i,v in embedded_user_ids_map.items()}
    
        
    # Be careful, anything we do that df = assigns into the df, or ['creates'] a new column
    # will result in a new object, besides the original when in a for loop!!!
    for df in df_train, df_valid:
        # 0 = PAD/NA, 1 = STARTER, 2=False, 3=True
        # TODO: Keep on for tito and tito+ cv
        df.user_id = df.user_id.map(embedded_user_ids_map).astype(np.uint32)
        
        df.prior_question_had_explanation = (df.prior_question_had_explanation.astype(np.float16).fillna(-2) + 2).astype(np.uint8)
        #df.prior_question_had_explanation = df.prior_question_had_explanation.astype(np.float16).fillna(0).astype(np.uint8)
        
        # NOTE: Pre-sorted the parquets:
        # df.sort_values(['user_id', 'timestamp'], inplace=True)
    
    # SPECIAL NOTE / TODO:
    # Valid set has some new users — about 1/3rd in our case
    # We want its user_ids to map to 0 = pad, to simulate us not
    # having any history for them. We do this in the dataloader
    # I guess....
    mask_user_ids = set(df_train.user_id.unique()) - set(df_valid.user_id.unique())
    
    
    gc.collect()
    
    
    #########################################
#     diagnostic_content_id_mean_map = df_train[['content_type_id', 'task_container_id', 'user_id', 'content_id', 'prior_question_had_explanation', 'answered_correctly']][
#         (df_train.content_type_id==0) &
#         (df_train.task_container_id<40)
#     ]

#     diagnostics_content_id = []
#     current_user_id = None
#     add = True
#     for user_id, content_id, prior_question_had_explanation in tqdm(diagnostic_content_id_mean_map[[
#         'user_id','content_id','prior_question_had_explanation'
#     ]].values): 
#         # run until we find prior_question_had_explanation
#         if current_user_id != user_id:
#             current_user_id = user_id
#             add = True

#         if prior_question_had_explanation==3:
#             add = False

#         if add:
#             diagnostics_content_id.append(content_id)

#     #10% of the population
#     diagnostics_content_id = pd.Series(diagnostics_content_id).value_counts()
#     diagnostics_content_id = diagnostics_content_id[diagnostics_content_id>40000].index.tolist()
#     diagnostic_content_id_mean_map = diagnostic_content_id_mean_map[['content_id','answered_correctly']][
#         diagnostic_content_id_mean_map.content_id.isin(diagnostics_content_id)
#     ].groupby('content_id').answered_correctly.mean().to_dict()
#     diagnostic_content_id_mean_map_values = list(diagnostic_content_id_mean_map.values())
    #########################################

    
    df_train.reset_index(inplace=True, drop=True)
    df_valid.reset_index(inplace=True, drop=True)
    
    # Cannot be in loop since we're creating a new feature:
    df_train.content_id += 1
    df_valid.content_id += 1
    
#     # NOTE: If we remove the dropping of bad_users w/ TCID count < 5
#     # Then this block can be removed and we can move the diagnostic building here
#     diagnostic_content_id_mean_map = {
#         # The mapping of the features
#         k+1:v for k,v in diagnostic_content_id_mean_map.items()
#     }
#     diagnostic_content_id_order_map = {
#         # The order of the features in our storage
#         k:i for i,k in enumerate(diagnostic_content_id_mean_map)
#     }
    
    # Fix lecture content_ids
    mask = (df_train.content_type_id == 1)
    df_train.loc[mask, 'content_id'] = df_train[mask].content_id.map(lecture_ids_map)
    df_train.loc[mask, 'answered_correctly'] = 1 # default for lectures, loss be masked anyway
    mask = (df_valid.content_type_id == 1)
    df_valid.loc[mask, 'content_id'] = df_valid[mask].content_id.map(lecture_ids_map)
    df_valid.loc[mask, 'answered_correctly'] = 1 # default for lectures, loss be masked anyway
    
    df_train['part_id'] = df_train.content_id.map(part_ids_map).astype(np.uint8)
    df_valid['part_id'] = df_valid.content_id.map(part_ids_map).astype(np.uint8)
    
    # Bundle IDs; 0=pad, 1=lecture
    bundle_id_map = {v:i+2 for i,v in enumerate(df_questions.bundle_id.unique())}
    df_train['bundle_id'] = df_train.content_id.map(bundle_id_map).fillna(0).astype(np.uint16)
    df_valid['bundle_id'] = df_valid.content_id.map(bundle_id_map).fillna(0).astype(np.uint16)
    
    #df_train['tag_group_id'] = df_train.content_id.map(tag_group_mapper).astype(np.uint16)
    #df_valid['tag_group_id'] = df_valid.content_id.map(tag_group_mapper).astype(np.uint16)
    
    ################################################################
    ## IMPORTANT : Question Incorrect Rank, we basically need this for the entire dataset
    ## And it should be calculated a-priori
    
    # How we calculated it:
#     question_incorrect_ranks = df[['content_id', 'user_answer', 'answered_correctly']]
#     question_incorrect_ranks = question_incorrect_ranks[question_incorrect_ranks.answered_correctly == 0]
#     question_incorrect_ranks = question_incorrect_ranks.groupby(['content_id','user_answer'], sort=False).count().reset_index()
#     question_incorrect_ranks.columns = ['content_id', 'user_answer', 'incorrect_rank']
    
#     # NOTE: We can also force ensure we have all {question_id,user_answer} pairs here............. so we dont have to fillna(0)....
#     # 0=pad, 1=starter, 2=correct, 3=wrong1, 4=wrong2, 5=wrong3
#     question_incorrect_ranks.sort_values(['content_id', 'incorrect_rank'], inplace=True)
#     question_incorrect_ranks['incorrect_rank'] = 3 + question_incorrect_ranks.groupby('content_id', sort=False).cumcount()

#     question_incorrect_ranks.content_id = question_incorrect_ranks.content_id.astype(np.int32)
#     question_incorrect_ranks.user_answer = question_incorrect_ranks.user_answer.astype(np.int8)
#     joblib.dump(question_incorrect_ranks, f'./question_incorrect_ranks.pkl')

    question_incorrect_ranks = joblib.load(f'./question_incorrect_ranks.pkl')
    
    df_train = df_train.merge(question_incorrect_ranks, how='left', on=['user_answer','content_id'])
    df_train.loc[df_train.incorrect_rank.isna(), 'incorrect_rank'] = 2 # fillna w/ correct
    df_train.incorrect_rank = df_train.incorrect_rank.astype(np.uint8)
    
    df_valid = df_valid.merge(question_incorrect_ranks, how='left', on=['user_answer','content_id'])
    df_valid.loc[df_valid.incorrect_rank.isna(), 'incorrect_rank'] = 2 # fillna w/ correct
    df_valid.incorrect_rank = df_valid.incorrect_rank.astype(np.uint8)
    ################################################################
    
    gc.collect()

    ###############################################################
    ######## IMPORTANT: NOTE: FOR VALID SET, WE SHOULD BE PROVIDING THE TRAINING SET HISTORY........
    ######## JUST LIKE IN INFERENCE; BUT IT SHOULDN'T COUNT.......... towards accuracy . loss
    # Sampling Strategy (for train)
    # Take first 500 records of all user ids in training data
    user_record_count = df_train.groupby('user_id', sort=False).size()

    # Scale the prior to make them sum 1 (probability distribution).
    user_record_count_priors = (user_record_count / user_record_count.sum())#.cumsum()
    Ntrn = user_record_count_priors.shape[0]

    num_slots = (user_record_count_priors / user_record_count_priors.min()).astype(np.uint16)
    train_user_id_sampling = np.concatenate([
        np.array([user_id] * slots, dtype=np.uint32)
        for user_id, slots in num_slots.to_dict().items()
    ])
    del user_record_count, user_record_count_priors, num_slots

    # Sampling Strategy (for valid)
    # Take first 500 records of all user ids in training data
    user_record_count = df_valid.groupby('user_id', sort=False).size()

    # Scale the prior to make them sum 1 (probability distribution).
    user_record_count_priors = (user_record_count / user_record_count.sum())#.cumsum()
    Nval = user_record_count_priors.shape[0]

    num_slots = (user_record_count_priors / user_record_count_priors.min()).astype(np.uint16)
    valid_user_id_sampling = np.concatenate([
        np.array([user_id] * slots, dtype=np.uint32)
        for user_id, slots in num_slots.to_dict().items()
    ])
    del user_record_count, user_record_count_priors, num_slots
    
    gc.collect()
    
    # Sample N ids with replacement with previous computed probabilities.
    # N in my case is the same as different user ids you have in the training data.
    # The result will have repeated ids.
    #samples = np.random.randint(0,user_id_sampling.shape[0], size=N)
    #samples = user_id_sampling[samples]
    ###############################################################
    
    user_features = defaultdict(newUserFeatures())
    
    # Index: 0 = per-ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3 
    session_content_num_u = np.zeros(df_train.shape[0], dtype=np.float32)
    session_duration_u    = np.zeros(df_train.shape[0], dtype=np.float64)
    session_correctness_u = np.zeros(df_train.shape[0], dtype=np.float32)
    session_ans_duration_u= np.zeros(df_train.shape[0], dtype=np.float64)
    lifetime_ans_duration_u= np.zeros(df_train.shape[0], dtype=np.float64)
    lag_ts_u_recency   = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency1 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency2 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency3 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_incorrect_ts_u_recency = np.zeros(df_train.shape[0], dtype=np.float64)
    last_correct_ts_u_recency   = np.zeros(df_train.shape[0], dtype=np.float64)
    correctness_u_recency       = np.zeros(df_train.shape[0], dtype=np.float32)
    part_correctness_u_recency  = np.zeros(df_train.shape[0], dtype=np.float32)
    diagnostic_u_recency        = np.zeros((df_train.shape[0],6), dtype=np.float32)
    answer_ratio1               = np.zeros(df_train.shape[0], dtype=np.float32)
    answer_ratio2               = np.zeros(df_train.shape[0], dtype=np.float32)
    correct_streak_u            = np.zeros(df_train.shape[0], dtype=np.uint16)
    incorrect_streak_u          = np.zeros(df_train.shape[0], dtype=np.uint16)
    correct_streak_alltime_u    = np.zeros(df_train.shape[0], dtype=np.uint16)
    incorrect_streak_alltime_u  = np.zeros(df_train.shape[0], dtype=np.uint16)
    encountered                 = np.zeros(df_train.shape[0], dtype=np.uint8)
    
    for idx, (_, user_id, content_id, task_container_id, content_type_id, part_id, timestamp, prior_question_elapsed_time, answered_correctly, user_answer) in enumerate(tqdm(df_train[[
        'user_id', 'content_id', 'task_container_id', 'content_type_id', 'part_id', 'timestamp', 'prior_question_elapsed_time', 'answered_correctly', 'user_answer'
    ]].itertuples())):
        # Adjustments
        part_id -= 1  # So we can index at 0
        (
            cached_task_container_id_u,
            task_container_num_questions_u,
            last_question_ts_u_recency2,
            session_u,
            last_ts_u,
            last_correct_ts_u,
            last_incorrect_ts_u,
            correctness_u,
            part_correctness_u,
            diagnostic_u_1,
            diagnostic_u_2,
            diagnostic_u_3,
            diagnostic_u_4,
            diagnostic_u_5,
            diagnostic_u_6,
            answer_ratio,
            streak_u,
            content_encounter,
        ) = user_features[user_id].get_features()
        part_correctness_u__part = part_correctness_u[part_id]
            
        # Step 1) Bundle Alignment operation - in kernel submission, this will run each time
        if cached_task_container_id_u[0] != task_container_id:
            # Initialize:
            cached_task_container_id_u[0] = task_container_id
            
            # Index: 0 = per-ts, 1 = per-bundle
            last_correct_ts_u[1]   = last_correct_ts_u[0]
            last_incorrect_ts_u[1] = last_incorrect_ts_u[0]

            # Index: 0,1 = per-ts, 2,3 = per-bundle. 0=correct qs, 1=total qs
            correctness_u[2] = correctness_u[0]
            correctness_u[3] = correctness_u[1]

            for part_id in range(7):
                # 7*4, [7parts][0=correct_ts,1=count_ts,2=correct_bundle,3=count_bundle]
                part_correctness_u[part_id][2] = part_correctness_u[part_id][0]
                part_correctness_u[part_id][3] = part_correctness_u[part_id][1]
            
            # Index: 0=current-ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3 
            # Order is important
            last_ts_u[3] = last_ts_u[2]
            last_ts_u[2] = last_ts_u[1]
            last_ts_u[1] = last_ts_u[0]
            last_ts_u[0] = timestamp
            
            # Streak
            streak_u[1] = streak_u[0]  # Update bundle
            streak_u[4] = streak_u[3]  # Update bundle
            if streak_u[2] < streak_u[1]: streak_u[2] = streak_u[1]  # Update all-time
            if streak_u[5] < streak_u[4]: streak_u[5] = streak_u[4]  # Update all-time
            
            # Answer Ratios
            answer_ratio[1] = answer_ratio[0]
            answer_ratio[3] = answer_ratio[2]
            answer_ratio[5] = answer_ratio[4]
            
            if task_container_id < 40:
                # Diagnostic Updates
                for i in range(2):
                    if np.isnan(diagnostic_u_1[i]): continue
                    diagnostic_u_1[i+2] = diagnostic_u_1[i]
                    diagnostic_u_1[i] = np.nan

                for i in range(6):
                    if np.isnan(diagnostic_u_2[i]): continue
                    diagnostic_u_2[i+6] = diagnostic_u_2[i]
                    diagnostic_u_2[i] = np.nan

                for i in range(3):
                    if np.isnan(diagnostic_u_3[i]): continue
                    diagnostic_u_3[i+3] = diagnostic_u_3[i]
                    diagnostic_u_3[i] = np.nan

                for i in range(5):
                    if not np.isnan(diagnostic_u_4[i]):
                        diagnostic_u_4[i+5] = diagnostic_u_4[i]
                        diagnostic_u_4[i] = np.nan
                    if not np.isnan(diagnostic_u_5[i]):
                        diagnostic_u_5[i+5] = diagnostic_u_5[i]
                        diagnostic_u_5[i] = np.nan

                for i in range(7):
                    if np.isnan(diagnostic_u_6[i]): continue
                    diagnostic_u_6[i+7] = diagnostic_u_6[i]
                    diagnostic_u_6[i] = np.nan
                    
                # The means:
                diagnostic_u_1[-1] = np.mean(diagnostic_u_1[2:-1])
                diagnostic_u_2[-1] = np.mean(diagnostic_u_2[6:-1])
                diagnostic_u_3[-1] = np.mean(diagnostic_u_3[3:-1])
                diagnostic_u_4[-1] = np.mean(diagnostic_u_4[5:-1])
                diagnostic_u_5[-1] = np.mean(diagnostic_u_5[5:-1])
                diagnostic_u_6[-1] = np.mean(diagnostic_u_6[7:-1])
                # End diagnostic features
            ###
            
            ###
            # Lag features
            if content_type_id==0:
                # We only reset for questions; lectures will ffill()
                task_container_num_questions_u[1] = task_container_num_questions_u[0]
                task_container_num_questions_u[0] = 0
                
                # Same claculation as last_ts_u_recency2
                last_question_ts_u_recency2[2] = last_question_ts_u_recency2[1]
                last_question_ts_u_recency2[1] = last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[0] = timestamp
                
                if not np.isnan(prior_question_elapsed_time):
                    session_u[5] += prior_question_elapsed_time * task_container_num_questions_u[1]
                    session_u[6] += prior_question_elapsed_time * task_container_num_questions_u[1]
            else:
                # Entering a lecture:
                diff = timestamp - last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[2] += diff
                last_question_ts_u_recency2[1] += diff
                last_question_ts_u_recency2[0] = timestamp
                # End lag features
                
            # Copy the session stuff over
            session_u[3] = session_u[1]
            session_u[4] = session_u[2]
            
            ###
        
        # Bake values - note, none of these change per task_container_id,
        # even though we're recalculating them all every iteration......
        lag_ts_u_recency[idx]            = last_question_ts_u_recency2[1] - last_question_ts_u_recency2[2] - prior_question_elapsed_time * task_container_num_questions_u[1]
        last_ts_u_recency1[idx]          = timestamp - last_ts_u[1]
        last_ts_u_recency2[idx]          = last_ts_u[1] - last_ts_u[2]
        last_ts_u_recency3[idx]          = last_ts_u[2] - last_ts_u[3]
        last_correct_ts_u_recency[idx]   = timestamp - last_correct_ts_u[1]
        last_incorrect_ts_u_recency[idx] = timestamp - last_incorrect_ts_u[1]
        correctness_u_recency[idx]       = correctness_u[2] / correctness_u[3]
        part_correctness_u_recency[idx]  = part_correctness_u__part[2] / part_correctness_u__part[3]
        session_content_num_u[idx]       = session_u[4] - 2 # we start at 6....
        session_duration_u[idx]          = 0 if np.isnan(session_u[0]) else timestamp - session_u[0]
        session_correctness_u[idx]       = session_u[3] / session_u[4]
        session_ans_duration_u[idx]      = session_u[5]
        lifetime_ans_duration_u[idx]     = session_u[6]
        diagnostic_u_recency[idx,0]      = diagnostic_u_1[-1]
        diagnostic_u_recency[idx,1]      = diagnostic_u_2[-1]
        diagnostic_u_recency[idx,2]      = diagnostic_u_3[-1]
        diagnostic_u_recency[idx,3]      = diagnostic_u_4[-1]
        diagnostic_u_recency[idx,4]      = diagnostic_u_5[-1]
        diagnostic_u_recency[idx,5]      = diagnostic_u_6[-1]
        answer_ratio1[idx]               = answer_ratio[1] / answer_ratio[5]
        answer_ratio2[idx]               = answer_ratio[3] / answer_ratio[5]
        correct_streak_u[idx]            = streak_u[1]
        incorrect_streak_u[idx]          = streak_u[4]
        correct_streak_alltime_u[idx]    = streak_u[2]
        incorrect_streak_alltime_u[idx]  = streak_u[5]
        
        
        # Step 0) Handle session stuff
        if np.isnan(last_ts_u[0]) or timestamp - last_ts_u[0] > 3600000 // 2:
            # Reset the session
            session_u[0] = timestamp
            session_u[1] = 2 # correct
            session_u[2] = 3 # count
            session_u[3] = 2 # correct
            session_u[4] = 3 # count
            session_u[5] = 0 # CumAnsTimeSessionNoLeak
        
        if content_encounter[content_id] == 1:
            encountered[idx] = 2
        else:
            # 0 for pad
            encountered[idx] = 1
        
        if content_type_id==0:
            # Lag: Increase for each question we see
            task_container_num_questions_u[0] += 1
            answer_ratio[5] += 1
            
            # Answer Selection Ratio for questions only
            if user_answer==1:
                answer_ratio[0] += 1
            elif user_answer==2:
                answer_ratio[2] += 1

        # Content Counters
        correctness_u[1] += 1
        part_correctness_u__part[1] += 1
        session_u[2] += 1
        content_encounter[content_id] = 1
        
        if answered_correctly==0:
            last_incorrect_ts_u[0] = timestamp
            streak_u[0] = 0  # reset correct counter
            streak_u[3] += 1 # increment incorrect counter
        else:
            last_correct_ts_u[0] = timestamp
            streak_u[0] += 1 # increment correct counter
            streak_u[3] = 0  # reset incorrect counter

            # Correct Content Counters
            correctness_u[0] += 1
            part_correctness_u__part[0] += 1
            session_u[1] += 1
            
        # Diagnostic Features
        if task_container_id < 40:
            if content_id == 6911:   diagnostic_u_1[0] = answered_correctly
            elif content_id == 7901: diagnostic_u_1[1] = answered_correctly
            elif content_id == 7219: diagnostic_u_2[0] = answered_correctly
            elif content_id == 6910: diagnostic_u_2[1] = answered_correctly
            elif content_id == 2066: diagnostic_u_2[2] = answered_correctly
            elif content_id == 6879: diagnostic_u_2[3] = answered_correctly
            elif content_id == 2065: diagnostic_u_2[4] = answered_correctly
            elif content_id == 1279: diagnostic_u_2[5] = answered_correctly
            elif content_id == 3365: diagnostic_u_3[0] = answered_correctly
            elif content_id == 4697: diagnostic_u_3[1] = answered_correctly
            elif content_id == 2594: diagnostic_u_3[2] = answered_correctly
            elif content_id == 6878: diagnostic_u_4[0] = answered_correctly
            elif content_id == 6912: diagnostic_u_4[1] = answered_correctly
            elif content_id == 4493: diagnostic_u_4[2] = answered_correctly
            elif content_id == 2596: diagnostic_u_4[3] = answered_correctly
            elif content_id == 7218: diagnostic_u_4[4] = answered_correctly
            elif content_id == 6909: diagnostic_u_5[0] = answered_correctly
            elif content_id == 7877: diagnostic_u_5[1] = answered_correctly
            elif content_id == 6880: diagnostic_u_5[2] = answered_correctly
            elif content_id == 7217: diagnostic_u_5[3] = answered_correctly
            elif content_id == 2595: diagnostic_u_5[4] = answered_correctly
            elif content_id == 2949: diagnostic_u_6[0] = answered_correctly
            elif content_id == 2948: diagnostic_u_6[1] = answered_correctly
            elif content_id == 4121: diagnostic_u_6[2] = answered_correctly
            elif content_id == 6881: diagnostic_u_6[3] = answered_correctly
            elif content_id == 6174: diagnostic_u_6[4] = answered_correctly
            elif content_id == 7220: diagnostic_u_6[5] = answered_correctly
            elif content_id == 3364: diagnostic_u_6[6] = answered_correctly

    df_train['lag_ts_u_recency'] = np.log1p(lag_ts_u_recency.clip(0,86400000) / 1000)
    df_train['last_ts_u_recency1']  = np.log1p(last_ts_u_recency1.clip(650,1209600000))
    df_train['last_ts_u_recency2']  = np.log1p(last_ts_u_recency2.clip(650,1209600000))
    df_train['last_ts_u_recency3']  = np.log1p(last_ts_u_recency3.clip(650,1209600000))
    df_train['last_correct_ts_u_recency'] = np.log1p(last_correct_ts_u_recency.clip(650,1209600000))
    df_train['last_incorrect_ts_u_recency'] = np.log1p(last_incorrect_ts_u_recency.clip(650,1209600000))
    df_train['correctness_u_recency'] = correctness_u_recency
    df_train['part_correctness_u_recency']  = part_correctness_u_recency
    df_train['session_content_num_u'] = np.log1p(session_content_num_u)
    df_train['session_duration_u'] = np.log1p(session_duration_u.clip(4100,18000000))
    df_train['session_correctness_u'] = session_correctness_u
    df_train['session_ans_duration_u'] = np.log1p(session_ans_duration_u.clip(4100,18000000))
    df_train['lifetime_ans_duration_u'] = np.log1p(lifetime_ans_duration_u.clip(4100,None))
    df_train['diagnostic_u_recency_1'] = diagnostic_u_recency[:, 0]
    df_train['diagnostic_u_recency_2'] = diagnostic_u_recency[:, 1]
    df_train['diagnostic_u_recency_3'] = diagnostic_u_recency[:, 2]
    df_train['diagnostic_u_recency_4'] = diagnostic_u_recency[:, 3]
    df_train['diagnostic_u_recency_5'] = diagnostic_u_recency[:, 4]
    df_train['diagnostic_u_recency_6'] = diagnostic_u_recency[:, 5]
    df_train['answer_ratio1'] = answer_ratio1
    df_train['answer_ratio2'] = answer_ratio2
    df_train['correct_streak_u'] = np.log1p(correct_streak_u)
    df_train['incorrect_streak_u'] = np.log1p(incorrect_streak_u)
    df_train['correct_streak_alltime_u'] = np.log1p(correct_streak_alltime_u)
    df_train['incorrect_streak_alltime_u'] = np.log1p(incorrect_streak_alltime_u)
    df_train['encountered'] = encountered
    
    lag_ts_u_recency_mean, lag_ts_u_recency_std = df_train.lag_ts_u_recency.mean(), df_train.lag_ts_u_recency.std()
    last_ts_u_recency1_mean, last_ts_u_recency1_std = df_train.last_ts_u_recency1.mean(), df_train.last_ts_u_recency1.std() 
    last_ts_u_recency2_mean, last_ts_u_recency2_std = df_train.last_ts_u_recency2.mean(), df_train.last_ts_u_recency2.std() 
    last_ts_u_recency3_mean, last_ts_u_recency3_std = df_train.last_ts_u_recency3.mean(), df_train.last_ts_u_recency3.std() 
    last_incorrect_ts_u_recency_mean, last_incorrect_ts_u_recency_std = df_train.last_incorrect_ts_u_recency.mean(), df_train.last_incorrect_ts_u_recency.std() 
    last_correct_ts_u_recency_mean, last_correct_ts_u_recency_std     = df_train.last_correct_ts_u_recency.mean(), df_train.last_correct_ts_u_recency.std() 
    correctness_u_recency_mean, correctness_u_recency_std             = df_train.correctness_u_recency.mean(), df_train.correctness_u_recency.std() 
    part_correctness_u_recency_mean, part_correctness_u_recency_std   = df_train.part_correctness_u_recency.mean(), df_train.part_correctness_u_recency.std()
    session_content_num_u_mean, session_content_num_u_std = df_train.session_content_num_u.mean(), df_train.session_content_num_u.std()
    session_duration_u_mean, session_duration_u_std = df_train.session_duration_u.mean(), df_train.session_duration_u.std()
    session_correctness_u_mean, session_correctness_u_std = df_train.session_correctness_u.mean(), df_train.session_correctness_u.std()    
    session_ans_duration_u_mean, session_ans_duration_u_std = df_train.session_ans_duration_u.mean(), df_train.session_ans_duration_u.std()
    lifetime_ans_duration_u_mean, lifetime_ans_duration_u_std = df_train.lifetime_ans_duration_u.mean(), df_train.lifetime_ans_duration_u.std()
    answer_ratio1_mean, answer_ratio1_std = df_train.answer_ratio1.mean(), df_train.answer_ratio1.std()
    answer_ratio2_mean, answer_ratio2_std = df_train.answer_ratio2.mean(), df_train.answer_ratio2.std()
    correct_streak_u_mean, correct_streak_u_std = df_train.correct_streak_u.mean(), df_train.correct_streak_u.std()
    incorrect_streak_u_mean, incorrect_streak_u_std = df_train.incorrect_streak_u.mean(), df_train.incorrect_streak_u.std()
    correct_streak_alltime_u_mean, correct_streak_alltime_u_std = df_train.correct_streak_alltime_u.mean(), df_train.correct_streak_alltime_u.std()
    incorrect_streak_alltime_u_mean, incorrect_streak_alltime_u_std = df_train.incorrect_streak_alltime_u.mean(), df_train.incorrect_streak_alltime_u.std()
    diagnostic_u_recency_1_mean, diagnostic_u_recency_1_std = df_train.diagnostic_u_recency_1.mean(), df_train.diagnostic_u_recency_1.std()
    diagnostic_u_recency_2_mean, diagnostic_u_recency_2_std = df_train.diagnostic_u_recency_2.mean(), df_train.diagnostic_u_recency_2.std()
    diagnostic_u_recency_3_mean, diagnostic_u_recency_3_std = df_train.diagnostic_u_recency_3.mean(), df_train.diagnostic_u_recency_3.std()
    diagnostic_u_recency_4_mean, diagnostic_u_recency_4_std = df_train.diagnostic_u_recency_4.mean(), df_train.diagnostic_u_recency_4.std()
    diagnostic_u_recency_5_mean, diagnostic_u_recency_5_std = df_train.diagnostic_u_recency_5.mean(), df_train.diagnostic_u_recency_5.std()
    diagnostic_u_recency_6_mean, diagnostic_u_recency_6_std = df_train.diagnostic_u_recency_6.mean(), df_train.diagnostic_u_recency_6.std()
    
    df_train.lag_ts_u_recency = ((df_train.lag_ts_u_recency   - lag_ts_u_recency_mean)  / lag_ts_u_recency_std).astype(np.float32) 
    df_train.last_ts_u_recency1 = ((df_train.last_ts_u_recency1   - last_ts_u_recency1_mean)  / last_ts_u_recency1_std).astype(np.float32) 
    df_train.last_ts_u_recency2 = ((df_train.last_ts_u_recency2   - last_ts_u_recency2_mean)  / last_ts_u_recency2_std).astype(np.float32) 
    df_train.last_ts_u_recency3 = ((df_train.last_ts_u_recency3   - last_ts_u_recency3_mean)  / last_ts_u_recency3_std).astype(np.float32) 
    df_train.last_incorrect_ts_u_recency = ((df_train.last_incorrect_ts_u_recency - last_incorrect_ts_u_recency_mean) / last_incorrect_ts_u_recency_std).astype(np.float32)
    df_train.last_correct_ts_u_recency   = ((df_train.last_correct_ts_u_recency   - last_correct_ts_u_recency_mean  ) / last_correct_ts_u_recency_std).astype(np.float32) 
    df_train.correctness_u_recency       = ((df_train.correctness_u_recency       - correctness_u_recency_mean      ) / correctness_u_recency_std).astype(np.float32) 
    df_train.part_correctness_u_recency  = ((df_train.part_correctness_u_recency  - part_correctness_u_recency_mean ) / part_correctness_u_recency_std).astype(np.float32)     
    df_train.session_content_num_u  = ((df_train.session_content_num_u   - session_content_num_u_mean )  / session_content_num_u_std).astype(np.float32) 
    df_train.session_duration_u     = ((df_train.session_duration_u      - session_duration_u_mean )     / session_duration_u_std).astype(np.float32) 
    df_train.session_correctness_u  = ((df_train.session_correctness_u   - session_correctness_u_mean )  / session_correctness_u_std).astype(np.float32) 
    df_train.session_ans_duration_u = ((df_train.session_ans_duration_u  - session_ans_duration_u_mean ) / session_ans_duration_u_std).astype(np.float32) 
    df_train.lifetime_ans_duration_u= ((df_train.lifetime_ans_duration_u - lifetime_ans_duration_u_mean) / lifetime_ans_duration_u_std).astype(np.float32) 
    df_train.diagnostic_u_recency_1 = ((df_train.diagnostic_u_recency_1  - diagnostic_u_recency_1_mean ) / diagnostic_u_recency_1_std).astype(np.float32) 
    df_train.diagnostic_u_recency_2 = ((df_train.diagnostic_u_recency_2  - diagnostic_u_recency_2_mean ) / diagnostic_u_recency_2_std).astype(np.float32) 
    df_train.diagnostic_u_recency_3 = ((df_train.diagnostic_u_recency_3  - diagnostic_u_recency_3_mean ) / diagnostic_u_recency_3_std).astype(np.float32) 
    df_train.diagnostic_u_recency_4 = ((df_train.diagnostic_u_recency_4  - diagnostic_u_recency_4_mean ) / diagnostic_u_recency_4_std).astype(np.float32) 
    df_train.diagnostic_u_recency_5 = ((df_train.diagnostic_u_recency_5  - diagnostic_u_recency_5_mean ) / diagnostic_u_recency_5_std).astype(np.float32) 
    df_train.diagnostic_u_recency_6 = ((df_train.diagnostic_u_recency_6  - diagnostic_u_recency_6_mean ) / diagnostic_u_recency_6_std).astype(np.float32) 
    df_train.answer_ratio1 = ((df_train.answer_ratio1  - answer_ratio1_mean ) / answer_ratio1_std).astype(np.float32) 
    df_train.answer_ratio2 = ((df_train.answer_ratio2  - answer_ratio2_mean ) / answer_ratio2_std).astype(np.float32) 
    df_train.correct_streak_u = ((df_train.correct_streak_u  - correct_streak_u_mean ) / correct_streak_u_std).astype(np.float32) 
    df_train.incorrect_streak_u = ((df_train.incorrect_streak_u  - incorrect_streak_u_mean ) / incorrect_streak_u_std).astype(np.float32) 
    df_train.correct_streak_alltime_u = ((df_train.correct_streak_alltime_u  - correct_streak_alltime_u_mean ) / correct_streak_alltime_u_std).astype(np.float32) 
    df_train.incorrect_streak_alltime_u = ((df_train.incorrect_streak_alltime_u  - incorrect_streak_alltime_u_mean ) / incorrect_streak_alltime_u_std).astype(np.float32) 

    
    df_train.loc[df_train.lag_ts_u_recency.isna(), 'lag_ts_u_recency'] = 0
    df_train.loc[df_train.last_ts_u_recency1.isna(), 'last_ts_u_recency1'] = 0
    df_train.loc[df_train.last_ts_u_recency2.isna(), 'last_ts_u_recency2'] = 0
    df_train.loc[df_train.last_ts_u_recency3.isna(), 'last_ts_u_recency3'] = 0
    df_train.loc[df_train.last_incorrect_ts_u_recency.isna(), 'last_incorrect_ts_u_recency'] = 0
    df_train.loc[df_train.last_correct_ts_u_recency.isna(), 'last_correct_ts_u_recency'] = 0
    df_train.loc[df_train.correctness_u_recency.isna(), 'correctness_u_recency'] = 0.653417715747257 # mean answered correctly
    df_train.loc[df_train.part_correctness_u_recency.isna(), 'part_correctness_u_recency'] = 0.653417715747257 # mean answered correctly across dset 
    df_train.loc[df_train.session_content_num_u.isna(), 'session_content_num_u'] = 0
    df_train.loc[df_train.session_duration_u.isna(), 'session_duration_u'] = 0
    df_train.loc[df_train.session_correctness_u.isna(), 'session_correctness_u'] = 0.653417715747257
    df_train.loc[df_train.session_ans_duration_u.isna(), 'session_ans_duration_u'] = 0
    df_train.loc[df_train.lifetime_ans_duration_u.isna(), 'lifetime_ans_duration_u'] = 0
    df_train.loc[df_train.diagnostic_u_recency_1.isna(), 'diagnostic_u_recency_1'] = 0
    df_train.loc[df_train.diagnostic_u_recency_2.isna(), 'diagnostic_u_recency_2'] = 0
    df_train.loc[df_train.diagnostic_u_recency_3.isna(), 'diagnostic_u_recency_3'] = 0
    df_train.loc[df_train.diagnostic_u_recency_4.isna(), 'diagnostic_u_recency_4'] = 0
    df_train.loc[df_train.diagnostic_u_recency_5.isna(), 'diagnostic_u_recency_5'] = 0
    df_train.loc[df_train.diagnostic_u_recency_6.isna(), 'diagnostic_u_recency_6'] = 0
    df_train.loc[df_train.answer_ratio1.isna(), 'answer_ratio1'] = 0
    df_train.loc[df_train.answer_ratio2.isna(), 'answer_ratio2'] = 0
    df_train.loc[df_train.correct_streak_u.isna(), 'correct_streak_u'] = 0
    df_train.loc[df_train.incorrect_streak_u.isna(), 'incorrect_streak_u'] = 0
    df_train.loc[df_train.correct_streak_alltime_u.isna(), 'correct_streak_alltime_u'] = 0
    df_train.loc[df_train.incorrect_streak_alltime_u.isna(), 'incorrect_streak_alltime_u'] = 0
    
    ###
    
    # For val set, we continue adding onto existing dictionaries
    # But set the vectors = 0
    session_content_num_u = np.zeros(df_valid.shape[0], dtype=np.float32)
    session_duration_u    = np.zeros(df_valid.shape[0], dtype=np.float64)
    session_correctness_u = np.zeros(df_valid.shape[0], dtype=np.float32)
    session_ans_duration_u= np.zeros(df_valid.shape[0], dtype=np.float64)
    lifetime_ans_duration_u= np.zeros(df_valid.shape[0], dtype=np.float64)
    lag_ts_u_recency   = np.zeros(df_valid.shape[0], dtype=np.float64)
    last_ts_u_recency1 = np.zeros(df_valid.shape[0], dtype=np.float64)
    last_ts_u_recency2 = np.zeros(df_valid.shape[0], dtype=np.float64)
    last_ts_u_recency3 = np.zeros(df_valid.shape[0], dtype=np.float64)
    last_incorrect_ts_u_recency = np.zeros(df_valid.shape[0], dtype=np.float64)
    last_correct_ts_u_recency   = np.zeros(df_valid.shape[0], dtype=np.float64)
    correctness_u_recency       = np.zeros(df_valid.shape[0], dtype=np.float32)
    part_correctness_u_recency  = np.zeros(df_valid.shape[0], dtype=np.float32)
    diagnostic_u_recency        = np.zeros((df_valid.shape[0],6), dtype=np.float32)
    answer_ratio1               = np.zeros(df_valid.shape[0], dtype=np.float32)
    answer_ratio2               = np.zeros(df_valid.shape[0], dtype=np.float32)
    correct_streak_u            = np.zeros(df_valid.shape[0], dtype=np.uint16)
    incorrect_streak_u          = np.zeros(df_valid.shape[0], dtype=np.uint16)
    correct_streak_alltime_u    = np.zeros(df_valid.shape[0], dtype=np.uint16)
    incorrect_streak_alltime_u  = np.zeros(df_valid.shape[0], dtype=np.uint16)
    encountered                 = np.zeros(df_valid.shape[0], dtype=np.uint8)
    
    for idx, (_, user_id, content_id, task_container_id, content_type_id, part_id, timestamp, prior_question_elapsed_time, answered_correctly, user_answer) in enumerate(tqdm(df_valid[[ 
        'user_id', 'content_id', 'task_container_id', 'content_type_id', 'part_id', 'timestamp', 'prior_question_elapsed_time', 'answered_correctly', 'user_answer'
    ]].itertuples())):
        # Adjustments
        part_id -= 1  # So we can index at 0
        (
            cached_task_container_id_u,
            task_container_num_questions_u,
            last_question_ts_u_recency2,
            session_u,
            last_ts_u,
            last_correct_ts_u,
            last_incorrect_ts_u,
            correctness_u,
            part_correctness_u,
            diagnostic_u_1,
            diagnostic_u_2,
            diagnostic_u_3,
            diagnostic_u_4,
            diagnostic_u_5,
            diagnostic_u_6,
            answer_ratio,
            streak_u,
            content_encounter,
        ) = user_features[user_id].get_features()
        part_correctness_u__part = part_correctness_u[part_id]
            
        # Step 1) Bundle Alignment operation - in kernel submission, this will run each time
        if cached_task_container_id_u[0] != task_container_id:
            # Initialize:
            cached_task_container_id_u[0] = task_container_id
            
            # Index: 0 = per-ts, 1 = per-bundle
            last_correct_ts_u[1]   = last_correct_ts_u[0]
            last_incorrect_ts_u[1] = last_incorrect_ts_u[0]

            # Index: 0,1 = per-ts, 2,3 = per-bundle. 0=correct qs, 1=total qs
            correctness_u[2] = correctness_u[0]
            correctness_u[3] = correctness_u[1]

            for part_id in range(7):
                # 7*4, [7parts][0=correct_ts,1=count_ts,2=correct_bundle,3=count_bundle]
                part_correctness_u[part_id][2] = part_correctness_u[part_id][0]
                part_correctness_u[part_id][3] = part_correctness_u[part_id][1]
            
            # Index: 0=current-ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3 
            # Order is important
            last_ts_u[3] = last_ts_u[2]
            last_ts_u[2] = last_ts_u[1]
            last_ts_u[1] = last_ts_u[0]
            last_ts_u[0] = timestamp
            
            # Streak
            streak_u[1] = streak_u[0]  # Update bundle
            streak_u[4] = streak_u[3]  # Update bundle
            if streak_u[2] < streak_u[1]: streak_u[2] = streak_u[1]  # Update all-time
            if streak_u[5] < streak_u[4]: streak_u[5] = streak_u[4]  # Update all-time
            
            # Answer Ratios
            answer_ratio[1] = answer_ratio[0]
            answer_ratio[3] = answer_ratio[2]
            answer_ratio[5] = answer_ratio[4]
            
            if task_container_id < 40:
                # Diagnostic Updates
                for i in range(2):
                    if np.isnan(diagnostic_u_1[i]): continue
                    diagnostic_u_1[i+2] = diagnostic_u_1[i]
                    diagnostic_u_1[i] = np.nan

                for i in range(6):
                    if np.isnan(diagnostic_u_2[i]): continue
                    diagnostic_u_2[i+6] = diagnostic_u_2[i]
                    diagnostic_u_2[i] = np.nan

                for i in range(3):
                    if np.isnan(diagnostic_u_3[i]): continue
                    diagnostic_u_3[i+3] = diagnostic_u_3[i]
                    diagnostic_u_3[i] = np.nan

                for i in range(5):
                    if not np.isnan(diagnostic_u_4[i]):
                        diagnostic_u_4[i+5] = diagnostic_u_4[i]
                        diagnostic_u_4[i] = np.nan
                    if not np.isnan(diagnostic_u_5[i]):
                        diagnostic_u_5[i+5] = diagnostic_u_5[i]
                        diagnostic_u_5[i] = np.nan

                for i in range(7):
                    if np.isnan(diagnostic_u_6[i]): continue
                    diagnostic_u_6[i+7] = diagnostic_u_6[i]
                    diagnostic_u_6[i] = np.nan
                    
                # The means:
                diagnostic_u_1[-1] = np.mean(diagnostic_u_1[2:-1])
                diagnostic_u_2[-1] = np.mean(diagnostic_u_2[6:-1])
                diagnostic_u_3[-1] = np.mean(diagnostic_u_3[3:-1])
                diagnostic_u_4[-1] = np.mean(diagnostic_u_4[5:-1])
                diagnostic_u_5[-1] = np.mean(diagnostic_u_5[5:-1])
                diagnostic_u_6[-1] = np.mean(diagnostic_u_6[7:-1])
                # End diagnostic features
            ###
            
            ###
            # Lag features
            if content_type_id==0:
                # We only reset for questions; lectures will ffill()
                task_container_num_questions_u[1] = task_container_num_questions_u[0]
                task_container_num_questions_u[0] = 0
                
                # Same claculation as last_ts_u_recency2
                last_question_ts_u_recency2[2] = last_question_ts_u_recency2[1]
                last_question_ts_u_recency2[1] = last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[0] = timestamp
                
                if not np.isnan(prior_question_elapsed_time):
                    session_u[5] += prior_question_elapsed_time * task_container_num_questions_u[1]
                    session_u[6] += prior_question_elapsed_time * task_container_num_questions_u[1]
            else:
                # Entering a lecture:
                diff = timestamp - last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[2] += diff
                last_question_ts_u_recency2[1] += diff
                last_question_ts_u_recency2[0] = timestamp
                # End lag features
                
            # Copy the session stuff over
            session_u[3] = session_u[1]
            session_u[4] = session_u[2]
            
            ###
        
        # Bake values - note, none of these change per task_container_id,
        # even though we're recalculating them all every iteration......
        lag_ts_u_recency[idx]            = last_question_ts_u_recency2[1] - last_question_ts_u_recency2[2] - prior_question_elapsed_time * task_container_num_questions_u[1]
        last_ts_u_recency1[idx]          = timestamp - last_ts_u[1]
        last_ts_u_recency2[idx]          = last_ts_u[1] - last_ts_u[2]
        last_ts_u_recency3[idx]          = last_ts_u[2] - last_ts_u[3]
        last_correct_ts_u_recency[idx]   = timestamp - last_correct_ts_u[1]
        last_incorrect_ts_u_recency[idx] = timestamp - last_incorrect_ts_u[1]
        correctness_u_recency[idx]       = correctness_u[2] / correctness_u[3]
        part_correctness_u_recency[idx]  = part_correctness_u__part[2] / part_correctness_u__part[3]
        session_content_num_u[idx]       = session_u[4] - 2 # we start at 6....
        session_duration_u[idx]          = 0 if np.isnan(session_u[0]) else timestamp - session_u[0]
        session_correctness_u[idx]       = session_u[3] / session_u[4]
        session_ans_duration_u[idx]      = session_u[5]
        lifetime_ans_duration_u[idx]     = session_u[6]
        diagnostic_u_recency[idx,0]      = diagnostic_u_1[-1]
        diagnostic_u_recency[idx,1]      = diagnostic_u_2[-1]
        diagnostic_u_recency[idx,2]      = diagnostic_u_3[-1]
        diagnostic_u_recency[idx,3]      = diagnostic_u_4[-1]
        diagnostic_u_recency[idx,4]      = diagnostic_u_5[-1]
        diagnostic_u_recency[idx,5]      = diagnostic_u_6[-1]
        answer_ratio1[idx]               = answer_ratio[1] / answer_ratio[5]
        answer_ratio2[idx]               = answer_ratio[3] / answer_ratio[5]
        correct_streak_u[idx]            = streak_u[1]
        incorrect_streak_u[idx]          = streak_u[4]
        correct_streak_alltime_u[idx]    = streak_u[2]
        incorrect_streak_alltime_u[idx]  = streak_u[5]
        
        
        # Step 0) Handle session stuff
        if np.isnan(last_ts_u[0]) or timestamp - last_ts_u[0] > 3600000 // 2:
            # Reset the session
            session_u[0] = timestamp
            session_u[1] = 2 # correct
            session_u[2] = 3 # count
            session_u[3] = 2 # correct
            session_u[4] = 3 # count
            session_u[5] = 0 # CumAnsTimeSessionNoLeak
        
        if content_encounter[content_id] == 1:
            encountered[idx] = 2
        else:
            # 0 for pad
            encountered[idx] = 1
        
        if content_type_id==0:
            # Lag: Increase for each question we see
            task_container_num_questions_u[0] += 1
            answer_ratio[5] += 1
            
            # Answer Selection Ratio for questions only
            if user_answer==1:
                answer_ratio[0] += 1
            elif user_answer==2:
                answer_ratio[2] += 1

        # Content Counters
        correctness_u[1] += 1
        part_correctness_u__part[1] += 1
        session_u[2] += 1
        content_encounter[content_id] = 1
        
        if answered_correctly==0:
            last_incorrect_ts_u[0] = timestamp
            streak_u[0] = 0  # reset correct counter
            streak_u[3] += 1 # increment incorrect counter
        else:
            last_correct_ts_u[0] = timestamp
            streak_u[0] += 1 # increment correct counter
            streak_u[3] = 0  # reset incorrect counter

            # Correct Content Counters
            correctness_u[0] += 1
            part_correctness_u__part[0] += 1
            session_u[1] += 1
            
        # Diagnostic Features
        if task_container_id < 40:
            if content_id == 6911:   diagnostic_u_1[0] = answered_correctly
            elif content_id == 7901: diagnostic_u_1[1] = answered_correctly
            elif content_id == 7219: diagnostic_u_2[0] = answered_correctly
            elif content_id == 6910: diagnostic_u_2[1] = answered_correctly
            elif content_id == 2066: diagnostic_u_2[2] = answered_correctly
            elif content_id == 6879: diagnostic_u_2[3] = answered_correctly
            elif content_id == 2065: diagnostic_u_2[4] = answered_correctly
            elif content_id == 1279: diagnostic_u_2[5] = answered_correctly
            elif content_id == 3365: diagnostic_u_3[0] = answered_correctly
            elif content_id == 4697: diagnostic_u_3[1] = answered_correctly
            elif content_id == 2594: diagnostic_u_3[2] = answered_correctly
            elif content_id == 6878: diagnostic_u_4[0] = answered_correctly
            elif content_id == 6912: diagnostic_u_4[1] = answered_correctly
            elif content_id == 4493: diagnostic_u_4[2] = answered_correctly
            elif content_id == 2596: diagnostic_u_4[3] = answered_correctly
            elif content_id == 7218: diagnostic_u_4[4] = answered_correctly
            elif content_id == 6909: diagnostic_u_5[0] = answered_correctly
            elif content_id == 7877: diagnostic_u_5[1] = answered_correctly
            elif content_id == 6880: diagnostic_u_5[2] = answered_correctly
            elif content_id == 7217: diagnostic_u_5[3] = answered_correctly
            elif content_id == 2595: diagnostic_u_5[4] = answered_correctly
            elif content_id == 2949: diagnostic_u_6[0] = answered_correctly
            elif content_id == 2948: diagnostic_u_6[1] = answered_correctly
            elif content_id == 4121: diagnostic_u_6[2] = answered_correctly
            elif content_id == 6881: diagnostic_u_6[3] = answered_correctly
            elif content_id == 6174: diagnostic_u_6[4] = answered_correctly
            elif content_id == 7220: diagnostic_u_6[5] = answered_correctly
            elif content_id == 3364: diagnostic_u_6[6] = answered_correctly

    
    # No need to recalculate mean, std - just use train sets
    df_valid['lag_ts_u_recency'] = np.log1p(lag_ts_u_recency.clip(0,86400000) / 1000)
    df_valid['last_ts_u_recency1']  = np.log1p(last_ts_u_recency1.clip(650,1209600000))
    df_valid['last_ts_u_recency2']  = np.log1p(last_ts_u_recency2.clip(650,1209600000))
    df_valid['last_ts_u_recency3']  = np.log1p(last_ts_u_recency3.clip(650,1209600000))
    df_valid['last_correct_ts_u_recency'] = np.log1p(last_correct_ts_u_recency.clip(650,1209600000))
    df_valid['last_incorrect_ts_u_recency'] = np.log1p(last_incorrect_ts_u_recency.clip(650,1209600000))
    df_valid['correctness_u_recency'] = correctness_u_recency
    df_valid['part_correctness_u_recency']  = part_correctness_u_recency
    df_valid['session_content_num_u'] = np.log1p(session_content_num_u)
    df_valid['session_duration_u'] = np.log1p(session_duration_u.clip(4100,18000000))
    df_valid['session_correctness_u'] = session_correctness_u
    df_valid['session_ans_duration_u'] = np.log1p(session_ans_duration_u.clip(4100,18000000))
    df_valid['lifetime_ans_duration_u'] = np.log1p(lifetime_ans_duration_u.clip(4100,None))
    df_valid['diagnostic_u_recency_1'] = diagnostic_u_recency[:, 0]
    df_valid['diagnostic_u_recency_2'] = diagnostic_u_recency[:, 1]
    df_valid['diagnostic_u_recency_3'] = diagnostic_u_recency[:, 2]
    df_valid['diagnostic_u_recency_4'] = diagnostic_u_recency[:, 3]
    df_valid['diagnostic_u_recency_5'] = diagnostic_u_recency[:, 4]
    df_valid['diagnostic_u_recency_6'] = diagnostic_u_recency[:, 5]
    df_valid['answer_ratio1'] = answer_ratio1
    df_valid['answer_ratio2'] = answer_ratio2
    df_valid['correct_streak_u'] = np.log1p(correct_streak_u)
    df_valid['incorrect_streak_u'] = np.log1p(incorrect_streak_u)
    df_valid['correct_streak_alltime_u'] = np.log1p(correct_streak_alltime_u)
    df_valid['incorrect_streak_alltime_u'] = np.log1p(incorrect_streak_alltime_u)
    df_valid['encountered'] = encountered
    
    df_valid.lag_ts_u_recency = ((df_valid.lag_ts_u_recency   - lag_ts_u_recency_mean)  / lag_ts_u_recency_std).astype(np.float32) 
    df_valid.last_ts_u_recency1 = ((df_valid.last_ts_u_recency1   - last_ts_u_recency1_mean)  / last_ts_u_recency1_std).astype(np.float32) 
    df_valid.last_ts_u_recency2 = ((df_valid.last_ts_u_recency2   - last_ts_u_recency2_mean)  / last_ts_u_recency2_std).astype(np.float32) 
    df_valid.last_ts_u_recency3 = ((df_valid.last_ts_u_recency3   - last_ts_u_recency3_mean)  / last_ts_u_recency3_std).astype(np.float32) 
    df_valid.last_incorrect_ts_u_recency = ((df_valid.last_incorrect_ts_u_recency - last_incorrect_ts_u_recency_mean) / last_incorrect_ts_u_recency_std).astype(np.float32)
    df_valid.last_correct_ts_u_recency   = ((df_valid.last_correct_ts_u_recency   - last_correct_ts_u_recency_mean  ) / last_correct_ts_u_recency_std).astype(np.float32) 
    df_valid.correctness_u_recency       = ((df_valid.correctness_u_recency       - correctness_u_recency_mean      ) / correctness_u_recency_std).astype(np.float32) 
    df_valid.part_correctness_u_recency  = ((df_valid.part_correctness_u_recency  - part_correctness_u_recency_mean ) / part_correctness_u_recency_std).astype(np.float32)     
    df_valid.session_content_num_u  = ((df_valid.session_content_num_u   - session_content_num_u_mean )  / session_content_num_u_std).astype(np.float32) 
    df_valid.session_duration_u     = ((df_valid.session_duration_u      - session_duration_u_mean )     / session_duration_u_std).astype(np.float32) 
    df_valid.session_correctness_u  = ((df_valid.session_correctness_u   - session_correctness_u_mean )  / session_correctness_u_std).astype(np.float32) 
    df_valid.session_ans_duration_u = ((df_valid.session_ans_duration_u  - session_ans_duration_u_mean ) / session_ans_duration_u_std).astype(np.float32) 
    df_valid.lifetime_ans_duration_u= ((df_valid.lifetime_ans_duration_u - lifetime_ans_duration_u_mean) / lifetime_ans_duration_u_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_1 = ((df_valid.diagnostic_u_recency_1  - diagnostic_u_recency_1_mean ) / diagnostic_u_recency_1_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_2 = ((df_valid.diagnostic_u_recency_2  - diagnostic_u_recency_2_mean ) / diagnostic_u_recency_2_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_3 = ((df_valid.diagnostic_u_recency_3  - diagnostic_u_recency_3_mean ) / diagnostic_u_recency_3_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_4 = ((df_valid.diagnostic_u_recency_4  - diagnostic_u_recency_4_mean ) / diagnostic_u_recency_4_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_5 = ((df_valid.diagnostic_u_recency_5  - diagnostic_u_recency_5_mean ) / diagnostic_u_recency_5_std).astype(np.float32) 
    df_valid.diagnostic_u_recency_6 = ((df_valid.diagnostic_u_recency_6  - diagnostic_u_recency_6_mean ) / diagnostic_u_recency_6_std).astype(np.float32) 
    df_valid.answer_ratio1 = ((df_valid.answer_ratio1  - answer_ratio1_mean ) / answer_ratio1_std).astype(np.float32) 
    df_valid.answer_ratio2 = ((df_valid.answer_ratio2  - answer_ratio2_mean ) / answer_ratio2_std).astype(np.float32) 
    df_valid.correct_streak_u = ((df_valid.correct_streak_u  - correct_streak_u_mean ) / correct_streak_u_std).astype(np.float32) 
    df_valid.incorrect_streak_u = ((df_valid.incorrect_streak_u  - incorrect_streak_u_mean ) / incorrect_streak_u_std).astype(np.float32) 
    df_valid.correct_streak_alltime_u = ((df_valid.correct_streak_alltime_u  - correct_streak_alltime_u_mean ) / correct_streak_alltime_u_std).astype(np.float32) 
    df_valid.incorrect_streak_alltime_u = ((df_valid.incorrect_streak_alltime_u  - incorrect_streak_alltime_u_mean ) / incorrect_streak_alltime_u_std).astype(np.float32) 
    
    df_valid.loc[df_valid.lag_ts_u_recency.isna(), 'lag_ts_u_recency'] = 0
    df_valid.loc[df_valid.last_ts_u_recency1.isna(), 'last_ts_u_recency1'] = 0
    df_valid.loc[df_valid.last_ts_u_recency2.isna(), 'last_ts_u_recency2'] = 0
    df_valid.loc[df_valid.last_ts_u_recency3.isna(), 'last_ts_u_recency3'] = 0
    df_valid.loc[df_valid.last_incorrect_ts_u_recency.isna(), 'last_incorrect_ts_u_recency'] = 0
    df_valid.loc[df_valid.last_correct_ts_u_recency.isna(), 'last_correct_ts_u_recency'] = 0
    df_valid.loc[df_valid.correctness_u_recency.isna(), 'correctness_u_recency'] = 0.653417715747257 # mean answered correctly
    df_valid.loc[df_valid.part_correctness_u_recency.isna(), 'part_correctness_u_recency'] = 0.653417715747257 # mean answered correctly across dset TODO This is probably wrong
    df_valid.loc[df_valid.session_content_num_u.isna(), 'session_content_num_u'] = 0
    df_valid.loc[df_valid.session_duration_u.isna(), 'session_duration_u'] = 0
    df_valid.loc[df_valid.session_correctness_u.isna(), 'session_correctness_u'] = 0.653417715747257
    df_valid.loc[df_valid.session_ans_duration_u.isna(), 'session_ans_duration_u'] = 0
    df_valid.loc[df_valid.lifetime_ans_duration_u.isna(), 'lifetime_ans_duration_u'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_1.isna(), 'diagnostic_u_recency_1'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_2.isna(), 'diagnostic_u_recency_2'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_3.isna(), 'diagnostic_u_recency_3'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_4.isna(), 'diagnostic_u_recency_4'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_5.isna(), 'diagnostic_u_recency_5'] = 0
    df_valid.loc[df_valid.diagnostic_u_recency_6.isna(), 'diagnostic_u_recency_6'] = 0
    df_valid.loc[df_valid.answer_ratio1.isna(), 'answer_ratio1'] = 0
    df_valid.loc[df_valid.answer_ratio2.isna(), 'answer_ratio2'] = 0
    df_valid.loc[df_valid.correct_streak_u.isna(), 'correct_streak_u'] = 0
    df_valid.loc[df_valid.incorrect_streak_u.isna(), 'incorrect_streak_u'] = 0
    df_valid.loc[df_valid.correct_streak_alltime_u.isna(), 'correct_streak_alltime_u'] = 0
    df_valid.loc[df_valid.incorrect_streak_alltime_u.isna(), 'incorrect_streak_alltime_u'] = 0
    
    ################################################################
    for df in df_train, df_valid:
        # This part is important though:
        # Convert to seconds for embedding, then mean fill across entire datasert
        # 0 = pad/NA
        # 1 = starter
        # 2 = ....
        #df.prior_question_elapsed_time.fillna(MEAN_prior_question_elapsed_time, inplace=True)
        df.prior_question_elapsed_time //= 1000 # for embedding 0-300, looks already capped
        df.loc[df.prior_question_elapsed_time.isna(), 'prior_question_elapsed_time'] = -2
        df.prior_question_elapsed_time += 2
        
    ###############################################################
    # Since there are users in train that are also in val,
    # we need to extend into val the train indices (historical data).
    # But we only want to _evaluate_ on the new valid portion.
    # How the hell we do that? For now, we'll just keep them entirely separate.
    train_prev_user_id, train_user_id_indices = None, {}
    
    for idx, user_id in enumerate(tqdm(df_train.user_id.values)):
        if train_prev_user_id != user_id:
            if train_prev_user_id != None:
                # End: Not inclusive
                train_user_id_indices[train_prev_user_id].append(idx)

            # Start: Inclusive
            train_user_id_indices[user_id] = [idx]
            train_prev_user_id = user_id
            
    train_user_id_indices[train_prev_user_id].append(df_train.shape[0])
    

    valid_prev_user_id, valid_user_id_indices = None, {}
    for idx, user_id in enumerate(tqdm(df_valid.user_id.values)):
        if valid_prev_user_id != user_id:
            if valid_prev_user_id != None:
                # End: Not inclusive
                valid_user_id_indices[valid_prev_user_id].append(idx)

            # Start: Inclusive
            valid_user_id_indices[user_id] = [idx]
            valid_prev_user_id = user_id

    valid_user_id_indices[valid_prev_user_id].append(df_valid.shape[0])
    ###############################################################
    
    df_train = df_train[dataset_cols]
    df_valid = df_valid[dataset_cols]
    
    print(time()-t, 'seconds')
    return (
        df_train, df_valid, Ntrn, Nval,
        embedded_user_ids_map, mask_user_ids, part_ids_map, lecture_ids_map, bundle_id_map,
        train_user_id_sampling, valid_user_id_sampling,
        train_user_id_indices, valid_user_id_indices,
        question_incorrect_ranks,
        answer_ratio1_mean, answer_ratio1_std,
        answer_ratio2_mean, answer_ratio2_std,
        correct_streak_u_mean, correct_streak_u_std,
        incorrect_streak_u_mean, incorrect_streak_u_std,
        correct_streak_alltime_u_mean, correct_streak_alltime_u_std,
        incorrect_streak_alltime_u_mean, incorrect_streak_alltime_u_std,
        diagnostic_u_recency_1_mean, diagnostic_u_recency_1_std,
        diagnostic_u_recency_2_mean, diagnostic_u_recency_2_std,
        diagnostic_u_recency_3_mean, diagnostic_u_recency_3_std,
        diagnostic_u_recency_4_mean, diagnostic_u_recency_4_std,
        diagnostic_u_recency_5_mean, diagnostic_u_recency_5_std,
        diagnostic_u_recency_6_mean, diagnostic_u_recency_6_std,
        session_content_num_u_mean, session_content_num_u_std,
        session_duration_u_mean, session_duration_u_std,
        session_correctness_u_mean, session_correctness_u_std,
        session_ans_duration_u_mean, session_ans_duration_u_std,
        lifetime_ans_duration_u_mean, lifetime_ans_duration_u_std,
        correctness_u_recency_mean, correctness_u_recency_std,
        part_correctness_u_recency_mean, part_correctness_u_recency_std,
        lag_ts_u_recency_mean, lag_ts_u_recency_std,
        last_ts_u_recency1_mean, last_ts_u_recency1_std,
        last_ts_u_recency2_mean, last_ts_u_recency2_std,
        last_ts_u_recency3_mean, last_ts_u_recency3_std,
        last_incorrect_ts_u_recency_mean, last_incorrect_ts_u_recency_std,
        last_correct_ts_u_recency_mean, last_correct_ts_u_recency_std,
    )

In [ ]:
def newUserFeatures(): return UserFeatures()
def newUserDF(): return UserDF()
def prepare_sub_data(fold_number):
    # TODO: Create the prior_elapsed_time_cont feature!!!!!!!
    
    # TODO: Load up our mappings.
    # For ensemble, unfortunatley we have to use the same fold, no time for dual feature standarizations...
    
    # user_features = joblib.load('../input/uthman-riiid/user_features_full.jlib')
    # user_states = joblib.load('../input/uthman-riiid/user_states_full.jlib') # last 128 records dataset format
    t = time()

    # TODO: Test with a small one first...    
    df_train = pd.read_parquet(f'../input/cv-strategy-in-the-kaggle-environment/cv{fold_number+1}_train_sorted.parquet')[TRAIN_DTYPES.keys()]
    df_valid = pd.read_parquet(f'../input/cv-strategy-in-the-kaggle-environment/cv{fold_number+1}_valid_sorted.parquet')[TRAIN_DTYPES.keys()]    
    df_train = df_train.append(df_valid, sort=False)
    df_train.reset_index(inplace=True, drop=True)
    del df_valid
    df_train.sort_values(['user_id','timestamp'], inplace=True)
    
    # Optional
    #df_train.user_id = df_train.user_id.map(embedded_user_ids_map).astype(np.uint32)
    df_train.prior_question_had_explanation = (df_train.prior_question_had_explanation.astype(np.float16).fillna(-2) + 2).astype(np.uint8)
    
    df_train.content_id += 1
    
    mask = (df_train.content_type_id == 1)
    df_train.loc[mask, 'content_id'] = df_train[mask].content_id.map(lecture_ids_map)
    df_train.loc[mask, 'answered_correctly'] = 1 # default for lectures, loss be masked anyway
    
    df_train['part_id'] = df_train.content_id.map(part_ids_map).astype(np.uint8)
    df_train['bundle_id'] = df_train.content_id.map(bundle_id_map).fillna(0).astype(np.uint16)
    #question_incorrect_ranks = joblib.load(f'./question_incorrect_ranks.pkl')
    #df_train = df_train.merge(question_incorrect_ranks, how='left', on=['user_answer','content_id'])
    #df_train.loc[df_train.incorrect_rank.isna(), 'incorrect_rank'] = 2 # fillna w/ correct
    #df_train.incorrect_rank = df_train.incorrect_rank.astype(np.uint8)
    
    gc.collect()
    
    user_features = defaultdict(newUserFeatures)
    
    # Index: 0 = per-ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3 
    session_content_num_u = np.zeros(df_train.shape[0], dtype=np.float32)
    session_duration_u    = np.zeros(df_train.shape[0], dtype=np.float64)
    session_correctness_u = np.zeros(df_train.shape[0], dtype=np.float32)
    session_ans_duration_u= np.zeros(df_train.shape[0], dtype=np.float64)
    lifetime_ans_duration_u= np.zeros(df_train.shape[0], dtype=np.float64)
    lag_ts_u_recency   = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency1 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency2 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_ts_u_recency3 = np.zeros(df_train.shape[0], dtype=np.float64)
    last_incorrect_ts_u_recency = np.zeros(df_train.shape[0], dtype=np.float64)
    last_correct_ts_u_recency   = np.zeros(df_train.shape[0], dtype=np.float64)
    correctness_u_recency       = np.zeros(df_train.shape[0], dtype=np.float32)
    part_correctness_u_recency  = np.zeros(df_train.shape[0], dtype=np.float32)
    diagnostic_u_recency        = np.zeros((df_train.shape[0],6), dtype=np.float32)
    answer_ratio1               = np.zeros(df_train.shape[0], dtype=np.float32)
    answer_ratio2               = np.zeros(df_train.shape[0], dtype=np.float32)
    correct_streak_u            = np.zeros(df_train.shape[0], dtype=np.uint16)
    incorrect_streak_u          = np.zeros(df_train.shape[0], dtype=np.uint16)
    correct_streak_alltime_u    = np.zeros(df_train.shape[0], dtype=np.uint16)
    incorrect_streak_alltime_u  = np.zeros(df_train.shape[0], dtype=np.uint16)
    encountered                 = np.zeros(df_train.shape[0], dtype=np.uint8)
    
    for idx, (_, user_id, content_id, task_container_id, content_type_id, part_id, timestamp, prior_question_elapsed_time, answered_correctly, user_answer) in enumerate(tqdm(df_train[[
        'user_id', 'content_id', 'task_container_id', 'content_type_id', 'part_id', 'timestamp', 'prior_question_elapsed_time', 'answered_correctly', 'user_answer'
    ]].itertuples())):
        # Adjustments
        part_id -= 1  # So we can index at 0
        (
            cached_task_container_id_u,
            task_container_num_questions_u,
            last_question_ts_u_recency2,
            session_u,
            last_ts_u,
            last_correct_ts_u,
            last_incorrect_ts_u,
            correctness_u,
            part_correctness_u,
            diagnostic_u_1,
            diagnostic_u_2,
            diagnostic_u_3,
            diagnostic_u_4,
            diagnostic_u_5,
            diagnostic_u_6,
            answer_ratio,
            streak_u,
            content_encounter,
        ) = user_features[user_id].get_features()
        part_correctness_u__part = part_correctness_u[part_id]
            
        # Step 1) Bundle Alignment operation - in kernel submission, this will run each time
        if cached_task_container_id_u[0] != task_container_id:
            # Initialize:
            cached_task_container_id_u[0] = task_container_id
            
            # Index: 0 = per-ts, 1 = per-bundle
            last_correct_ts_u[1]   = last_correct_ts_u[0]
            last_incorrect_ts_u[1] = last_incorrect_ts_u[0]

            # Index: 0,1 = per-ts, 2,3 = per-bundle. 0=correct qs, 1=total qs
            correctness_u[2] = correctness_u[0]
            correctness_u[3] = correctness_u[1]

            for part_id in range(7):
                # 7*4, [7parts][0=correct_ts,1=count_ts,2=correct_bundle,3=count_bundle]
                part_correctness_u[part_id][2] = part_correctness_u[part_id][0]
                part_correctness_u[part_id][3] = part_correctness_u[part_id][1]
            
            # Index: 0=current-ts, 1 = per-bundle1, 2 = per-bundle2, 3 = per-bundle3 
            # Order is important
            last_ts_u[3] = last_ts_u[2]
            last_ts_u[2] = last_ts_u[1]
            last_ts_u[1] = last_ts_u[0]
            last_ts_u[0] = timestamp
            
            # Streak
            streak_u[1] = streak_u[0]  # Update bundle
            streak_u[4] = streak_u[3]  # Update bundle
            if streak_u[2] < streak_u[1]: streak_u[2] = streak_u[1]  # Update all-time
            if streak_u[5] < streak_u[4]: streak_u[5] = streak_u[4]  # Update all-time
            
            # Answer Ratios
            answer_ratio[1] = answer_ratio[0]
            answer_ratio[3] = answer_ratio[2]
            answer_ratio[5] = answer_ratio[4]
            
            if task_container_id < 40:
                # Diagnostic Updates
                for i in range(2):
                    if np.isnan(diagnostic_u_1[i]): continue
                    diagnostic_u_1[i+2] = diagnostic_u_1[i]
                    diagnostic_u_1[i] = np.nan

                for i in range(6):
                    if np.isnan(diagnostic_u_2[i]): continue
                    diagnostic_u_2[i+6] = diagnostic_u_2[i]
                    diagnostic_u_2[i] = np.nan

                for i in range(3):
                    if np.isnan(diagnostic_u_3[i]): continue
                    diagnostic_u_3[i+3] = diagnostic_u_3[i]
                    diagnostic_u_3[i] = np.nan

                for i in range(5):
                    if not np.isnan(diagnostic_u_4[i]):
                        diagnostic_u_4[i+5] = diagnostic_u_4[i]
                        diagnostic_u_4[i] = np.nan
                    if not np.isnan(diagnostic_u_5[i]):
                        diagnostic_u_5[i+5] = diagnostic_u_5[i]
                        diagnostic_u_5[i] = np.nan

                for i in range(7):
                    if np.isnan(diagnostic_u_6[i]): continue
                    diagnostic_u_6[i+7] = diagnostic_u_6[i]
                    diagnostic_u_6[i] = np.nan
                    
                # The means:
                diagnostic_u_1[-1] = np.mean(diagnostic_u_1[2:-1])
                diagnostic_u_2[-1] = np.mean(diagnostic_u_2[6:-1])
                diagnostic_u_3[-1] = np.mean(diagnostic_u_3[3:-1])
                diagnostic_u_4[-1] = np.mean(diagnostic_u_4[5:-1])
                diagnostic_u_5[-1] = np.mean(diagnostic_u_5[5:-1])
                diagnostic_u_6[-1] = np.mean(diagnostic_u_6[7:-1])
                # End diagnostic features
            ###
            
            ###
            # Lag features
            if content_type_id==0:
                # We only reset for questions; lectures will ffill()
                task_container_num_questions_u[1] = task_container_num_questions_u[0]
                task_container_num_questions_u[0] = 0
                
                # Same claculation as last_ts_u_recency2
                last_question_ts_u_recency2[2] = last_question_ts_u_recency2[1]
                last_question_ts_u_recency2[1] = last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[0] = timestamp
                
                if not np.isnan(prior_question_elapsed_time):
                    session_u[5] += prior_question_elapsed_time * task_container_num_questions_u[1]
                    session_u[6] += prior_question_elapsed_time * task_container_num_questions_u[1]
            else:
                # Entering a lecture:
                diff = timestamp - last_question_ts_u_recency2[0]
                last_question_ts_u_recency2[2] += diff
                last_question_ts_u_recency2[1] += diff
                last_question_ts_u_recency2[0] = timestamp
                # End lag features
                
            # Copy the session stuff over
            session_u[3] = session_u[1]
            session_u[4] = session_u[2]
            
            ###
        
        # Bake values - note, none of these change per task_container_id,
        # even though we're recalculating them all every iteration......
        lag_ts_u_recency[idx]            = last_question_ts_u_recency2[1] - last_question_ts_u_recency2[2] - prior_question_elapsed_time * task_container_num_questions_u[1]
        last_ts_u_recency1[idx]          = timestamp - last_ts_u[1]
        last_ts_u_recency2[idx]          = last_ts_u[1] - last_ts_u[2]
        last_ts_u_recency3[idx]          = last_ts_u[2] - last_ts_u[3]
        last_correct_ts_u_recency[idx]   = timestamp - last_correct_ts_u[1]
        last_incorrect_ts_u_recency[idx] = timestamp - last_incorrect_ts_u[1]
        correctness_u_recency[idx]       = correctness_u[2] / correctness_u[3]
        part_correctness_u_recency[idx]  = part_correctness_u__part[2] / part_correctness_u__part[3]
        session_content_num_u[idx]       = session_u[4] - 2 # we start at 6....
        session_duration_u[idx]          = 0 if np.isnan(session_u[0]) else timestamp - session_u[0]
        session_correctness_u[idx]       = session_u[3] / session_u[4]
        session_ans_duration_u[idx]      = session_u[5]
        lifetime_ans_duration_u[idx]     = session_u[6]
        diagnostic_u_recency[idx,0]      = diagnostic_u_1[-1]
        diagnostic_u_recency[idx,1]      = diagnostic_u_2[-1]
        diagnostic_u_recency[idx,2]      = diagnostic_u_3[-1]
        diagnostic_u_recency[idx,3]      = diagnostic_u_4[-1]
        diagnostic_u_recency[idx,4]      = diagnostic_u_5[-1]
        diagnostic_u_recency[idx,5]      = diagnostic_u_6[-1]
        answer_ratio1[idx]               = answer_ratio[1] / answer_ratio[5]
        answer_ratio2[idx]               = answer_ratio[3] / answer_ratio[5]
        correct_streak_u[idx]            = streak_u[1]
        incorrect_streak_u[idx]          = streak_u[4]
        correct_streak_alltime_u[idx]    = streak_u[2]
        incorrect_streak_alltime_u[idx]  = streak_u[5]
        
        
        # Step 0) Handle session stuff
        if np.isnan(last_ts_u[0]) or timestamp - last_ts_u[0] > 3600000 // 2:
            # Reset the session
            session_u[0] = timestamp
            session_u[1] = 2 # correct
            session_u[2] = 3 # count
            session_u[3] = 2 # correct
            session_u[4] = 3 # count
            session_u[5] = 0 # CumAnsTimeSessionNoLeak
        
        if content_encounter[content_id] == 1:
            encountered[idx] = 2
        else:
            # 0 for pad
            encountered[idx] = 1
        
        if content_type_id==0:
            # Lag: Increase for each question we see
            task_container_num_questions_u[0] += 1
            answer_ratio[5] += 1
            
            # Answer Selection Ratio for questions only
            if user_answer==1:
                answer_ratio[0] += 1
            elif user_answer==2:
                answer_ratio[2] += 1

        # Content Counters
        correctness_u[1] += 1
        part_correctness_u__part[1] += 1
        session_u[2] += 1
        content_encounter[content_id] = 1
        
        if answered_correctly==0:
            last_incorrect_ts_u[0] = timestamp
            streak_u[0] = 0  # reset correct counter
            streak_u[3] += 1 # increment incorrect counter
        else:
            last_correct_ts_u[0] = timestamp
            streak_u[0] += 1 # increment correct counter
            streak_u[3] = 0  # reset incorrect counter

            # Correct Content Counters
            correctness_u[0] += 1
            part_correctness_u__part[0] += 1
            session_u[1] += 1
            
        # Diagnostic Features
        if task_container_id < 40:
            if content_id == 6911:   diagnostic_u_1[0] = answered_correctly
            elif content_id == 7901: diagnostic_u_1[1] = answered_correctly
            elif content_id == 7219: diagnostic_u_2[0] = answered_correctly
            elif content_id == 6910: diagnostic_u_2[1] = answered_correctly
            elif content_id == 2066: diagnostic_u_2[2] = answered_correctly
            elif content_id == 6879: diagnostic_u_2[3] = answered_correctly
            elif content_id == 2065: diagnostic_u_2[4] = answered_correctly
            elif content_id == 1279: diagnostic_u_2[5] = answered_correctly
            elif content_id == 3365: diagnostic_u_3[0] = answered_correctly
            elif content_id == 4697: diagnostic_u_3[1] = answered_correctly
            elif content_id == 2594: diagnostic_u_3[2] = answered_correctly
            elif content_id == 6878: diagnostic_u_4[0] = answered_correctly
            elif content_id == 6912: diagnostic_u_4[1] = answered_correctly
            elif content_id == 4493: diagnostic_u_4[2] = answered_correctly
            elif content_id == 2596: diagnostic_u_4[3] = answered_correctly
            elif content_id == 7218: diagnostic_u_4[4] = answered_correctly
            elif content_id == 6909: diagnostic_u_5[0] = answered_correctly
            elif content_id == 7877: diagnostic_u_5[1] = answered_correctly
            elif content_id == 6880: diagnostic_u_5[2] = answered_correctly
            elif content_id == 7217: diagnostic_u_5[3] = answered_correctly
            elif content_id == 2595: diagnostic_u_5[4] = answered_correctly
            elif content_id == 2949: diagnostic_u_6[0] = answered_correctly
            elif content_id == 2948: diagnostic_u_6[1] = answered_correctly
            elif content_id == 4121: diagnostic_u_6[2] = answered_correctly
            elif content_id == 6881: diagnostic_u_6[3] = answered_correctly
            elif content_id == 6174: diagnostic_u_6[4] = answered_correctly
            elif content_id == 7220: diagnostic_u_6[5] = answered_correctly
            elif content_id == 3364: diagnostic_u_6[6] = answered_correctly

    df_train['lag_ts_u_recency'] = np.log1p(lag_ts_u_recency.clip(0,86400000) / 1000)
    df_train['last_ts_u_recency1']  = np.log1p(last_ts_u_recency1.clip(650,1209600000))
    df_train['last_ts_u_recency2']  = np.log1p(last_ts_u_recency2.clip(650,1209600000))
    df_train['last_ts_u_recency3']  = np.log1p(last_ts_u_recency3.clip(650,1209600000))
    df_train['last_correct_ts_u_recency'] = np.log1p(last_correct_ts_u_recency.clip(650,1209600000))
    df_train['last_incorrect_ts_u_recency'] = np.log1p(last_incorrect_ts_u_recency.clip(650,1209600000))
    df_train['correctness_u_recency'] = correctness_u_recency
    df_train['part_correctness_u_recency']  = part_correctness_u_recency
    df_train['session_content_num_u'] = np.log1p(session_content_num_u)
    df_train['session_duration_u'] = np.log1p(session_duration_u.clip(4100,18000000))
    df_train['session_correctness_u'] = session_correctness_u
    df_train['session_ans_duration_u'] = np.log1p(session_ans_duration_u.clip(4100,18000000))
    df_train['lifetime_ans_duration_u'] = np.log1p(lifetime_ans_duration_u.clip(4100,None))
    df_train['diagnostic_u_recency_1'] = diagnostic_u_recency[:, 0]
    df_train['diagnostic_u_recency_2'] = diagnostic_u_recency[:, 1]
    df_train['diagnostic_u_recency_3'] = diagnostic_u_recency[:, 2]
    df_train['diagnostic_u_recency_4'] = diagnostic_u_recency[:, 3]
    df_train['diagnostic_u_recency_5'] = diagnostic_u_recency[:, 4]
    df_train['diagnostic_u_recency_6'] = diagnostic_u_recency[:, 5]
    df_train['answer_ratio1'] = answer_ratio1
    df_train['answer_ratio2'] = answer_ratio2
    df_train['correct_streak_u'] = np.log1p(correct_streak_u)
    df_train['incorrect_streak_u'] = np.log1p(incorrect_streak_u)
    df_train['correct_streak_alltime_u'] = np.log1p(correct_streak_alltime_u)
    df_train['incorrect_streak_alltime_u'] = np.log1p(incorrect_streak_alltime_u)
    df_train['encountered'] = encountered
    
    df_train.lag_ts_u_recency = ((df_train.lag_ts_u_recency   - lag_ts_u_recency_mean)  / lag_ts_u_recency_std).astype(np.float32) 
    df_train.last_ts_u_recency1 = ((df_train.last_ts_u_recency1   - last_ts_u_recency1_mean)  / last_ts_u_recency1_std).astype(np.float32) 
    df_train.last_ts_u_recency2 = ((df_train.last_ts_u_recency2   - last_ts_u_recency2_mean)  / last_ts_u_recency2_std).astype(np.float32) 
    df_train.last_ts_u_recency3 = ((df_train.last_ts_u_recency3   - last_ts_u_recency3_mean)  / last_ts_u_recency3_std).astype(np.float32) 
    df_train.last_incorrect_ts_u_recency = ((df_train.last_incorrect_ts_u_recency - last_incorrect_ts_u_recency_mean) / last_incorrect_ts_u_recency_std).astype(np.float32)
    df_train.last_correct_ts_u_recency   = ((df_train.last_correct_ts_u_recency   - last_correct_ts_u_recency_mean  ) / last_correct_ts_u_recency_std).astype(np.float32) 
    df_train.correctness_u_recency       = ((df_train.correctness_u_recency       - correctness_u_recency_mean      ) / correctness_u_recency_std).astype(np.float32) 
    df_train.part_correctness_u_recency  = ((df_train.part_correctness_u_recency  - part_correctness_u_recency_mean ) / part_correctness_u_recency_std).astype(np.float32)     
    df_train.session_content_num_u  = ((df_train.session_content_num_u   - session_content_num_u_mean )  / session_content_num_u_std).astype(np.float32) 
    df_train.session_duration_u     = ((df_train.session_duration_u      - session_duration_u_mean )     / session_duration_u_std).astype(np.float32) 
    df_train.session_correctness_u  = ((df_train.session_correctness_u   - session_correctness_u_mean )  / session_correctness_u_std).astype(np.float32) 
    df_train.session_ans_duration_u = ((df_train.session_ans_duration_u  - session_ans_duration_u_mean ) / session_ans_duration_u_std).astype(np.float32) 
    df_train.lifetime_ans_duration_u= ((df_train.lifetime_ans_duration_u - lifetime_ans_duration_u_mean) / lifetime_ans_duration_u_std).astype(np.float32) 
    df_train.diagnostic_u_recency_1 = ((df_train.diagnostic_u_recency_1  - diagnostic_u_recency_1_mean ) / diagnostic_u_recency_1_std).astype(np.float32) 
    df_train.diagnostic_u_recency_2 = ((df_train.diagnostic_u_recency_2  - diagnostic_u_recency_2_mean ) / diagnostic_u_recency_2_std).astype(np.float32) 
    df_train.diagnostic_u_recency_3 = ((df_train.diagnostic_u_recency_3  - diagnostic_u_recency_3_mean ) / diagnostic_u_recency_3_std).astype(np.float32) 
    df_train.diagnostic_u_recency_4 = ((df_train.diagnostic_u_recency_4  - diagnostic_u_recency_4_mean ) / diagnostic_u_recency_4_std).astype(np.float32) 
    df_train.diagnostic_u_recency_5 = ((df_train.diagnostic_u_recency_5  - diagnostic_u_recency_5_mean ) / diagnostic_u_recency_5_std).astype(np.float32) 
    df_train.diagnostic_u_recency_6 = ((df_train.diagnostic_u_recency_6  - diagnostic_u_recency_6_mean ) / diagnostic_u_recency_6_std).astype(np.float32) 
    df_train.answer_ratio1 = ((df_train.answer_ratio1  - answer_ratio1_mean ) / answer_ratio1_std).astype(np.float32) 
    df_train.answer_ratio2 = ((df_train.answer_ratio2  - answer_ratio2_mean ) / answer_ratio2_std).astype(np.float32) 
    df_train.correct_streak_u = ((df_train.correct_streak_u  - correct_streak_u_mean ) / correct_streak_u_std).astype(np.float32) 
    df_train.incorrect_streak_u = ((df_train.incorrect_streak_u  - incorrect_streak_u_mean ) / incorrect_streak_u_std).astype(np.float32) 
    df_train.correct_streak_alltime_u = ((df_train.correct_streak_alltime_u  - correct_streak_alltime_u_mean ) / correct_streak_alltime_u_std).astype(np.float32) 
    df_train.incorrect_streak_alltime_u = ((df_train.incorrect_streak_alltime_u  - incorrect_streak_alltime_u_mean ) / incorrect_streak_alltime_u_std).astype(np.float32) 

    df_train.loc[df_train.lag_ts_u_recency.isna(), 'lag_ts_u_recency'] = 0
    df_train.loc[df_train.last_ts_u_recency1.isna(), 'last_ts_u_recency1'] = 0
    df_train.loc[df_train.last_ts_u_recency2.isna(), 'last_ts_u_recency2'] = 0
    df_train.loc[df_train.last_ts_u_recency3.isna(), 'last_ts_u_recency3'] = 0
    df_train.loc[df_train.last_incorrect_ts_u_recency.isna(), 'last_incorrect_ts_u_recency'] = 0
    df_train.loc[df_train.last_correct_ts_u_recency.isna(), 'last_correct_ts_u_recency'] = 0
    df_train.loc[df_train.correctness_u_recency.isna(), 'correctness_u_recency'] = 0.653417715747257 # mean answered correctly
    df_train.loc[df_train.part_correctness_u_recency.isna(), 'part_correctness_u_recency'] = 0.653417715747257 # mean answered correctly across dset 
    df_train.loc[df_train.session_content_num_u.isna(), 'session_content_num_u'] = 0
    df_train.loc[df_train.session_duration_u.isna(), 'session_duration_u'] = 0
    df_train.loc[df_train.session_correctness_u.isna(), 'session_correctness_u'] = 0.653417715747257
    df_train.loc[df_train.session_ans_duration_u.isna(), 'session_ans_duration_u'] = 0
    df_train.loc[df_train.lifetime_ans_duration_u.isna(), 'lifetime_ans_duration_u'] = 0
    df_train.loc[df_train.diagnostic_u_recency_1.isna(), 'diagnostic_u_recency_1'] = 0
    df_train.loc[df_train.diagnostic_u_recency_2.isna(), 'diagnostic_u_recency_2'] = 0
    df_train.loc[df_train.diagnostic_u_recency_3.isna(), 'diagnostic_u_recency_3'] = 0
    df_train.loc[df_train.diagnostic_u_recency_4.isna(), 'diagnostic_u_recency_4'] = 0
    df_train.loc[df_train.diagnostic_u_recency_5.isna(), 'diagnostic_u_recency_5'] = 0
    df_train.loc[df_train.diagnostic_u_recency_6.isna(), 'diagnostic_u_recency_6'] = 0
    df_train.loc[df_train.answer_ratio1.isna(), 'answer_ratio1'] = 0
    df_train.loc[df_train.answer_ratio2.isna(), 'answer_ratio2'] = 0
    df_train.loc[df_train.correct_streak_u.isna(), 'correct_streak_u'] = 0
    df_train.loc[df_train.incorrect_streak_u.isna(), 'incorrect_streak_u'] = 0
    df_train.loc[df_train.correct_streak_alltime_u.isna(), 'correct_streak_alltime_u'] = 0
    df_train.loc[df_train.incorrect_streak_alltime_u.isna(), 'incorrect_streak_alltime_u'] = 0
    
    df_train['prior_question_elapsed_time_cont'] = df_train.prior_question_elapsed_time
    df_train.prior_question_elapsed_time //= 1000 # for embedding 0-300, looks already capped
    df_train.loc[df_train.prior_question_elapsed_time.isna(), 'prior_question_elapsed_time'] = -2
    df_train.prior_question_elapsed_time += 2
    
    # Now, keep just the lasrt 128
    df_train = df_train.groupby('user_id', sort=False).tail(128)
    
    # Create a default dictionary for user dataframe cache
    user_dfs = defaultdict(newUserDF)
    
    # Save it out
    for idx, (_, user_id, task_container_id, content_id, part_id, prior_question_elapsed_time, prior_question_had_explanation, incorrect_rank, content_type_id, bundle_id, answer_ratio1, answer_ratio2, correct_streak_u, incorrect_streak_u, correct_streak_alltime_u, incorrect_streak_alltime_u, session_content_num_u, session_duration_u, session_ans_duration_u, lifetime_ans_duration_u, lag_ts_u_recency, last_ts_u_recency1, last_ts_u_recency2, last_ts_u_recency3, last_correct_ts_u_recency, last_incorrect_ts_u_recency, correctness_u_recency, part_correctness_u_recency, session_correctness_u, encountered, diagnostic_u_recency_1, diagnostic_u_recency_2, diagnostic_u_recency_3, diagnostic_u_recency_4, diagnostic_u_recency_5, diagnostic_u_recency_6, answered_correctly) \
    in enumerate(tqdm(df_train[dataset_cols[COL_OFFSET_CUT:]].itertuples())):
        state = user_dfs[user_id]
        
        state.lensize[0] += 1
        state.user_id.append(user_id)
        state.task_container_id.append(task_container_id)
        state.content_id.append(content_id)
        state.part_id.append(part_id)
        state.prior_question_elapsed_time.append(prior_question_elapsed_time)
        state.prior_question_had_explanation.append(prior_question_had_explanation)
        state.incorrect_rank.append(incorrect_rank) # decoder feature that needs help..........
        state.content_type_id.append(content_type_id)
        state.bundle_id.append(bundle_id)

        # only these of the 4 have signal (0-index)
        state.answer_ratio1.append(answer_ratio1)
        state.answer_ratio2.append(answer_ratio2)
        state.correct_streak_u.append(correct_streak_u)
        state.incorrect_streak_u.append(incorrect_streak_u)
        state.correct_streak_alltime_u.append(correct_streak_alltime_u)
        state.incorrect_streak_alltime_u.append(incorrect_streak_alltime_u)

        # our session information + a lifetime field (elapsed time sum on platform—probably should be capped)
        state.session_content_num_u.append(session_content_num_u)
        state.session_duration_u.append(session_duration_u)
        state.session_ans_duration_u.append(session_ans_duration_u)
        state.lifetime_ans_duration_u.append(lifetime_ans_duration_u)

        # First block of continuous - timestamp intervals
        state.lag_ts_u_recency.append(lag_ts_u_recency)
        state.last_ts_u_recency1.append(last_ts_u_recency1)
        state.last_ts_u_recency2.append(last_ts_u_recency2)
        state.last_ts_u_recency3.append(last_ts_u_recency3)
        state.last_correct_ts_u_recency.append(last_correct_ts_u_recency)
        state.last_incorrect_ts_u_recency.append(last_incorrect_ts_u_recency)

        # Second block of continuous - how often user is right on global/part/session basis
        state.correctness_u_recency.append(correctness_u_recency)
        state.part_correctness_u_recency.append(part_correctness_u_recency)
        state.session_correctness_u.append(session_correctness_u)

        # Have we been served this content before?
        state.encountered.append(encountered)

        # Average score of most frequent asked questions
        state.diagnostic_u_recency_1.append(diagnostic_u_recency_1)
        state.diagnostic_u_recency_2.append(diagnostic_u_recency_2)
        state.diagnostic_u_recency_3.append(diagnostic_u_recency_3)
        state.diagnostic_u_recency_4.append(diagnostic_u_recency_4)
        state.diagnostic_u_recency_5.append(diagnostic_u_recency_5)
        state.diagnostic_u_recency_6.append(diagnostic_u_recency_6)

        state.answered_correctly.append(answered_correctly)
    
    joblib.dump(user_features, './user_features_full.jlib')
    joblib.dump(user_dfs, './user_dfs_full.jlib')

In [ ]:
gc.collect()
prepare_sub_data(4)

# Dataset

In [ ]:
class Riiid(torch.utils.data.Dataset):
    def __init__(self, npdata, N, user_id_indices, user_id_sampling, mask_user_ids, augment=True):
        # df_train[dataset_cols[3:]].values
        self.npdata = npdata
        self.N = N
        self.user_id_indices = user_id_indices
        self.user_id_sampling = user_id_sampling
        self.CONTENT_MASK_TOKEN = 13941+2 - 1
        self.mask_user_ids = mask_user_ids
        self.augment = augment
        
    def __len__(self):
        return self.N
    
    def __getitem__(self, idx=None):
        # Take a random sequence for every id. It may lead to repeated sequences with
        # low probability. For example, if you have id 115 repeated 7 times, you will
        # take 7 random sequences for the user 115.
        cont_feats = [[0,0, 0,0, 0,0, 0,0, 0]]
        sess_feats = [[0,0,0,0]]
        diag_feats = [[0,0,0,0,0,0]]
        strk_feats = [[0,0,0,0,0,0]]
        
        # With replacement, so we ignore idx
        random_user_id = self.user_id_sampling[
            np.random.randint(0, self.user_id_sampling.shape[0])
        ]
        
        # NOTE:
        # These uesr ids are expected to already be mapped
        row_start, row_end = self.user_id_indices[random_user_id]
        row_len = row_end - row_start
        rows = self.npdata[row_start:row_end]
        
        if row_len <= WINDOW_SIZE:
            pad_len = WINDOW_SIZE - row_len
            padding = [PAD] * pad_len
            neg_pad = [-2] * pad_len

            # Augment indices order within Task Container ID, since it literally doesn't matter
            indices = np.arange(row_len, dtype=np.uint16)
            if self.augment:
                cuts = np.concatenate((
                    np.nonzero(np.concatenate([[1],np.diff(rows[:, COL_TASK_CONTAINER_ID])]))[0],
                    [row_len]
                ))
                indices = [ indices[cuts[i]:cuts[i+1]]  for i in range(len(cuts) - 1) ]
                for block in indices: np.random.shuffle(block)
                if len(indices) > 6:
                    ii = np.random.randint(0,len(indices)-1,(2,))
                    indices[ii[0]], indices[ii[0]+1] = indices[ii[0]+1], indices[ii[0]] 
                    indices[ii[1]], indices[ii[1]+1] = indices[ii[1]+1], indices[ii[1]] 
                indices = np.concatenate(indices)
            
            if pad_len==0:
                continuous = rows[indices, COL_LAG_TS_RECENCY:COL_SESSION_CORRECTNESS_U+1]
                session = rows[indices, COL_SESSION_CONTENT_NUM_U:COL_LIFETIME_ANS_DURATION_U+1]
                diagnostic = rows[indices, COL_DIAGNOSTIC_RECENCY_1:COL_DIAGNOSTIC_RECENCY_6+1]
                streak = rows[indices, COL_ANSWER_RATIO1:COL_INCORRECT_STREAK_ALLTIME_U+1]

            else:
                continuous = np.vstack((
                    rows[indices, COL_LAG_TS_RECENCY:COL_SESSION_CORRECTNESS_U+1],
                    cont_feats * pad_len
                ))
                
                session = np.vstack((
                    rows[indices, COL_SESSION_CONTENT_NUM_U:COL_LIFETIME_ANS_DURATION_U+1],
                    sess_feats * pad_len
                ))
                
                diagnostic = np.vstack((
                    rows[indices, COL_DIAGNOSTIC_RECENCY_1:COL_DIAGNOSTIC_RECENCY_6+1],
                    diag_feats * pad_len
                ))
                
                streak = np.vstack((
                    rows[indices, COL_ANSWER_RATIO1:COL_INCORRECT_STREAK_ALLTIME_U+1],
                    strk_feats * pad_len
                ))
            
            if self.mask_user_ids:
                user_ids = np.zeros(WINDOW_SIZE, dtype=int)
            else:
                user_ids = np.concatenate((rows[indices, COL_USER_ID], padding)).astype(int)
            
            return (
                idx,
                user_ids,
                np.concatenate((rows[indices, COL_CONTENT_ID], padding)).astype(int),
                np.concatenate((rows[indices, COL_CONTENT_TYPE_ID], padding)).astype(int),
                np.concatenate((rows[indices, COL_PART_ID], padding)).astype(int),
                np.concatenate((rows[indices, COL_ENCOUNTERED], padding)).astype(int),

                np.concatenate((
                    rows[indices, COL_PRIOR_QUESTION_ELAPSED_TIME],
                    padding
                )).astype(int),
                
                np.concatenate((
                    rows[indices, COL_PRIOR_QUESTION_EXPLANATION],
                    padding
                )).astype(int),
                
                # correctness_id
                np.concatenate((
                    [1], # starter
                    #2 + rows[:-1, COL_ANSWERED_CORRECTLY],
                    rows[indices[:-1], COL_INCORRECT_RANK],
                    padding
                )).astype(int),
                
                # Bundle
                np.concatenate((rows[indices, COL_BUNDLE_ID], padding)).astype(int),
                streak.astype(np.float32),
                
                # Continuous Features
                continuous.astype(np.float32),
                session.astype(np.float32),
                diagnostic.astype(np.float32),
                
                # padding mask:
                np.concatenate(([False]*(row_end - row_start), [True]*pad_len)).astype(np.uint8),
                
                # lecture mask
                np.concatenate((1==rows[indices, COL_CONTENT_TYPE_ID], [False]*pad_len)).astype(np.uint8),
                
                # labels:
                np.concatenate((rows[indices, COL_ANSWERED_CORRECTLY], padding)).astype(np.float32),
                
                # task container id:
                np.concatenate((rows[indices, COL_TASK_CONTAINER_ID], neg_pad)).astype(np.int),
            )
        
        # Return random WINDOW_SIZE crop from the range of this users inputs;
        # We also want later values to be samples more often that earlier samples
#         start_offset = max(
#             # No preference to further items, just let ranadomness do its thing with our data:
#             np.random.randint(0, row_len-WINDOW_SIZE),
#             #np.random.randint(0, row_len-WINDOW_SIZE),
#             #np.random.randint(0, row_len-WINDOW_SIZE),
#         )
        start_offset = np.random.randint(0, row_len-WINDOW_SIZE)

        indices = np.arange(start_offset, start_offset+WINDOW_SIZE, dtype=np.uint16)
        if self.augment:
            cuts = np.concatenate((
                np.nonzero(np.concatenate([[1],np.diff(rows[start_offset:start_offset+WINDOW_SIZE, COL_TASK_CONTAINER_ID])]))[0],
                [WINDOW_SIZE]
            ))
            indices = [ indices[cuts[i]:cuts[i+1]]  for i in range(len(cuts) - 1) ]
            for block in indices: np.random.shuffle(block)
            if len(indices) > 6:
                ii = np.random.randint(0,len(indices)-1,(2,))
                indices[ii[0]], indices[ii[0]+1] = indices[ii[0]+1], indices[ii[0]] 
                indices[ii[1]], indices[ii[1]+1] = indices[ii[1]+1], indices[ii[1]] 
            indices = np.concatenate(indices)
        
        if self.mask_user_ids:
            user_ids = np.zeros(WINDOW_SIZE, dtype=int)
        else:
            user_ids = rows[indices, COL_USER_ID].astype(int)
            
        return (
            idx,
            user_ids,
            rows[indices, COL_CONTENT_ID].astype(int),
            rows[indices, COL_CONTENT_TYPE_ID].astype(int),
            rows[indices, COL_PART_ID].astype(int),
            rows[indices, COL_ENCOUNTERED].astype(int),
            
            np.concatenate((
                #[1], # NEW Shifted decoder stuff:
                #rows[start_offset:start_offset+WINDOW_SIZE-1, COL_PRIOR_QUESTION_ELAPSED_TIME],
                rows[indices, COL_PRIOR_QUESTION_ELAPSED_TIME],
            )).astype(int),

            np.concatenate((
                #[1], # NEW Shifted decoder stuff:
                #rows[start_offset:start_offset+WINDOW_SIZE-1, COL_PRIOR_QUESTION_EXPLANATION],
                rows[indices, COL_PRIOR_QUESTION_EXPLANATION],
            )).astype(int),

            # correctness_id
            np.concatenate((
                [1], # starter
                rows[indices[:-1], COL_INCORRECT_RANK],
                #2+rows[start_offset:start_offset+WINDOW_SIZE-1, COL_ANSWERED_CORRECTLY],
            )).astype(int),
            
            # Bundle
            rows[indices, COL_BUNDLE_ID].astype(int),
            
            # STREAK:
            rows[indices, COL_ANSWER_RATIO1:COL_INCORRECT_STREAK_ALLTIME_U+1].astype(np.float32),
            
            # Continuous Features:
            rows[indices, COL_LAG_TS_RECENCY:COL_SESSION_CORRECTNESS_U+1].astype(np.float32),
            rows[indices, COL_SESSION_CONTENT_NUM_U:COL_LIFETIME_ANS_DURATION_U+1].astype(np.float32),
            rows[indices, COL_DIAGNOSTIC_RECENCY_1:COL_DIAGNOSTIC_RECENCY_6+1].astype(np.float32),
            
            # padding mask:
            np.array([False]*WINDOW_SIZE).astype(np.uint8),
            
            # lecture mask
            (1 == rows[indices, COL_CONTENT_TYPE_ID]).astype(np.uint8),

            # labels:
            rows[indices, COL_ANSWERED_CORRECTLY].astype(np.float32),
            
            # task container id:
            rows[indices, COL_TASK_CONTAINER_ID].astype(np.int),
        )

In [ ]:
def collate_fn(batch):
    (
        _, user_id, content_id, content_type_id, part_id, encountered_id, prior_question_elapsed_time,
        prior_question_had_explanation, correctness_id, bundle_id, cont_streaks,
        continuous, session, diagnostic, mask, lecture_mask, labels, task_container_id
    ) = zip(*batch)
        
    user_id = torch.LongTensor(user_id)
    content_id = torch.LongTensor(content_id)
    
    content_type_id = torch.LongTensor(content_type_id)
    part_id = torch.LongTensor(part_id)
    encountered_id  = torch.LongTensor(encountered_id)
    prior_question_elapsed_time = torch.LongTensor(prior_question_elapsed_time)
    prior_question_had_explanation = torch.LongTensor(prior_question_had_explanation)
    correctness_id = torch.LongTensor(correctness_id)
    bundle_id = torch.LongTensor(bundle_id)
    cont_streaks = torch.FloatTensor(cont_streaks)
    continuous = torch.FloatTensor(continuous)
    diagnostic = torch.FloatTensor(diagnostic)
    session = torch.FloatTensor(session)
    mask = torch.BoolTensor(mask)
    lecture_mask = torch.BoolTensor(lecture_mask)
    
    labels = torch.FloatTensor(labels)
    task_container_id = torch.LongTensor(task_container_id)
    
    # remember the order
    return (
        user_id, content_id, content_type_id, part_id, encountered_id,
        prior_question_elapsed_time, prior_question_had_explanation,
        correctness_id, bundle_id, cont_streaks,
        continuous, session, diagnostic, mask, lecture_mask, labels, task_container_id
    )



# Model

In [ ]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [ ]:
class ItemResponseTheory(nn.Module):
    '''
    chance = probability of random guess = 0.25
    a = concept ability
    B = exercise difficulty (can be the mean)
    '''
    def __init__(self, chance=0.25):
        super(ItemResponseTheory, self).__init__()
        self.chance = chance
        
    def forward(self, input, no_responses):
        # p(correct|a,B) = c + (1-c) / (1 + np.exp(B-a)), where:
        
        alpha = input[:,:,0]
        Beta  = input[:,:,1]
        
        no_responses = no_responses.permute(1,0)
        return 1/no_responses + (1-1/no_responses) / (1 + torch.exp(Beta-alpha))
        #return self.chance + (1-chance) / (1 + torch.exp(Beta-alpha))

In [ ]:
class BartSinusoidalPositionalEmbedding(nn.Embedding):
    """This module produces sinusoidal positional embeddings of any length."""

    def __init__(self, num_positions: int, embedding_dim: int, padding_idx=None):
        super().__init__(num_positions+1, embedding_dim)
        self.weight = self._init_weight(self.weight)
        self.positions = None
        
    @staticmethod
    def _init_weight(out: nn.Parameter):
        """
        Identical to the XLM create_sinusoidal_embeddings except features are not interleaved. The cos features are in
        the 2nd half of the vector. [dim // 2:]
        """
        n_pos, dim = out.shape
        position_enc = np.array(
            [[pos / np.power(10000, 2 * (j // 2) / dim) for j in range(dim)] for pos in range(n_pos)]
        )
        out.requires_grad = False  # set early to avoid an error in pytorch-1.8+
        sentinel = dim // 2 if dim % 2 == 0 else (dim // 2) + 1
        out[:, :sentinel] = torch.FloatTensor(np.sin(position_enc[:, 0::2]))
        out[:, sentinel:] = torch.FloatTensor(np.cos(position_enc[:, 1::2]))
        
        # Scale this bad boy
        out *= 0.1 # 10% of whatever it is
        
        out.detach_()
        return out

    @torch.no_grad()
    def forward(self, start:int=0):
        if self.positions is None:
            self.positions = torch.arange(
                0, WINDOW_SIZE+1,
                dtype=torch.long,
                device=self.weight.device
            )
            
        return super().forward(self.positions)[start:start+WINDOW_SIZE]

In [ ]:
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

In [ ]:
class TransposeBatchNorm(nn.Module):
    def __init__(self, num_features):
        super(TransposeBatchNorm, self).__init__()
        self.bn = nn.BatchNorm1d(num_features)
    def forward(self, x):
        return self.bn(x.transpose(1,2)).transpose(1,2)

In [ ]:
def generate_mask(size, diagonal=1):        
    return torch.triu(torch.ones(size, size)==1, diagonal=diagonal)

def tasks_mask(tasks, seq_length, diagonal=1):
    future_mask = generate_mask(seq_length, diagonal=diagonal).to(tasks.device)
    container_mask= torch.ones((seq_length, seq_length)).to(tasks.device)
    container_mask=(container_mask*tasks.reshape(1,-1))==(container_mask*tasks.reshape(-1,1))
    future_mask=future_mask+container_mask
    future_mask = future_mask.fill_diagonal_(False)
    return future_mask

def tasks_3d_mask(tasks, seq_length=WINDOW_SIZE, nhead=1, diagonal=1):
    #https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/206620
    mask_3d = [tasks_mask(t, seq_length, diagonal=diagonal) for t in tasks]
    mask_3d = torch.stack(mask_3d, dim=0)
    # Need BS*num_heads shape
    repeat_3d = [mask_3d for t in range(nhead)]
    repeat_3d = torch.cat(repeat_3d)
    return repeat_3d

In [ ]:
class ThinTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(ThinTransformerEncoderLayer, self).__init__()
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)


    def forward(self, src, src_mask = None, src_key_padding_mask = None):
        r"""Pass the input through the encoder layer.

        Args:
            src: the sequence to the encoder layer (required).
            src_mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).

        Shape:
            see the docs in Transformer class.
        """
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        
        # Mask Out:
        src[src_key_padding_mask.transpose(0,1)[:,:,None].expand(src.shape)] = 0
        return src

In [ ]:
class SaintTransformerModel(nn.Module):
    def __init__(
        self,
        d_model:int=32,
        nhead:int=2,
        num_users:int=393657+1,
        num_layers:int=2,
        num_exercises:int=13941+2,         # NOTE: 1+total number of QUESTIONS+LECTURES + Mask at the end
        num_answer_streaks:int=8+1,
        num_content_types:int=2,
        num_parts:int=7+1,                 # TOTAL
        num_bundles:int=9765+2,
        num_prior_question_elapsed_time:int=301+2,
        num_explanable:int=2+2,
        num_max_seq_len:int=WINDOW_SIZE,   # TODO, figure this out, do we want to extend past?
        num_encountered:int=2+1,
        dropout:float=0.1,
        emb_dropout:float=0.2,
        initrange:float = 0.02
    ):
        super(SaintTransformerModel, self).__init__()

        self.d_model = d_model
        self.nhead = nhead
        self.emb_dropout = nn.Dropout(p=emb_dropout)
        
#         # These two have double dropout, due to their cardinality:
#         self.user_embeddings = nn.Sequential(
#             nn.Embedding(num_users, 16, padding_idx=0),
#             #nn.Linear(16, d_model, bias=False),
#             #nn.Dropout(p=emb_dropout)
#         )
#         self.answer_streak_embeddings = nn.Embedding(num_answer_streaks, d_model, padding_idx=0)
        self.exercise_embeddings = nn.Embedding(num_exercises, d_model, padding_idx=0)
        self.bundle_embeddings = nn.Embedding(num_bundles, d_model, padding_idx=0)
        self.content_type_embeddings = nn.Embedding(num_content_types, d_model, padding_idx=0)
        self.sin_pos_embedder = BartSinusoidalPositionalEmbedding(num_positions=WINDOW_SIZE, embedding_dim=d_model, padding_idx=0) 
        self.part_embeddings = nn.Embedding(num_parts, d_model, padding_idx=0)
        self.prior_question_elapsed_time_embeddings = nn.Embedding(num_prior_question_elapsed_time, d_model, padding_idx=0) 
        self.prior_question_had_explanation_embeddings = nn.Embedding(num_explanable, d_model, padding_idx=0)
        self.correctness_embeddings = nn.Embedding(4+2, d_model, padding_idx=0)
        self.encountered_embeddings = nn.Embedding(num_encountered, d_model, padding_idx=0)

        self.cont_mlp = nn.Sequential(
            # LN ? BN
            nn.Linear(25, d_model//2),
            nn.Dropout(p=emb_dropout),
            nn.ReLU(inplace=True),
            nn.Linear(d_model//2, d_model, bias=False),
            nn.ReLU(inplace=True),
        )
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model, nhead, dim_feedforward=d_model,#2048,
                dropout=dropout, activation='relu'
            ),
            num_layers
        )
        
        self.transformer_history = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model, nhead*2, dim_feedforward=d_model,#2048,
                dropout=dropout, activation='relu'
            ),
            1 # history only has 1 layer
        )
        
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(
                d_model, nhead, dim_feedforward=d_model,#2048,
                dropout=dropout, activation='relu'
            ),
            num_layers
        )
        
        self.decoder = nn.Linear(d_model,1)
        self.init_weights(initrange)

    def init_weights(self, initrange:float = 0.02) -> None:
        self.content_type_embeddings.weight.data.normal_(0, initrange)
        self.encountered_embeddings.weight.data.normal_(0, initrange)
        self.exercise_embeddings.weight.data.normal_(0, initrange)
        self.bundle_embeddings.weight.data.normal_(0, initrange)
        self.part_embeddings.weight.data.normal_(0, initrange)
        self.prior_question_elapsed_time_embeddings.weight.data.normal_(0, initrange)
        self.prior_question_had_explanation_embeddings.weight.data.normal_(0, initrange)
        self.correctness_embeddings.weight.data.normal_(0, initrange)
        #self.decoder.bias.data.zero_()
        self.decoder.weight.data.normal_(0, initrange)
        
    @torch.cuda.amp.autocast()
    def forward(
        self,
        user_id, content_id, content_type_id, part_id, encountered_id,
        prior_question_elapsed_time, prior_question_had_explanation,
        correctness_id, bundle_id, cont_streaks,
        continuous, session, diagnostic, padding_mask, task_container_id
    ):
        batch_size = content_id.shape[0]
        seq_len = content_id.shape[-1]
        
        future_mask = torch.ones([seq_len, seq_len], device=task_container_id.device, dtype=torch.uint8)
        future_mask = future_mask.triu_(1).view(seq_len, seq_len).bool()
        history_mask = tasks_3d_mask(task_container_id, seq_length=WINDOW_SIZE, nhead=self.nhead*2)
        
        ##########
        embedded_src = self.emb_dropout(
            self.content_type_embeddings(content_type_id)
            + self.part_embeddings(part_id)
            + self.bundle_embeddings(bundle_id)
            + self.exercise_embeddings(content_id)
            + self.sin_pos_embedder(start=1)
        ).transpose(0, 1) # (S, N, E)
        
        output_src = self.emb_dropout(
            # Shifted by BUNDLE already
            self.prior_question_elapsed_time_embeddings(prior_question_elapsed_time)
            + self.prior_question_had_explanation_embeddings(prior_question_had_explanation)
            + self.encountered_embeddings(encountered_id)
            
            + self.cont_mlp(
                torch.cat([
                    cont_streaks,
                    diagnostic,
                    session,
                    continuous,
                ], dim=-1)
            )
            
            + self.sin_pos_embedder(start=0)
        ).transpose(0, 1) # (S, N, E)

        history_src = self.emb_dropout(
            self.correctness_embeddings(correctness_id)
            + self.sin_pos_embedder(start=1)
        ).transpose(0, 1)
        
        history = self.transformer_history(
            src=history_src,
            mask=history_mask,
            src_key_padding_mask=padding_mask,
        )
        with torch.no_grad():
            history[torch.isnan(history)] = 0

        memory = self.transformer_encoder(
            src=embedded_src,
            mask=future_mask,
            src_key_padding_mask=padding_mask,
        )

        output = self.transformer_decoder(
            # TODO: Try adding memory into here? so we have a full 'interaction' embedding
            tgt = output_src + history, #embedded_src
            memory = memory,
            tgt_mask = future_mask,
            memory_mask = future_mask,
            tgt_key_padding_mask = padding_mask,
            memory_key_padding_mask = padding_mask,
        )
        
        # TODO: Maybe we can concatenate instead of add...
        output = self.decoder(output).transpose(1, 0)
        return output

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, networks):
        super(SiameseNetwork, self).__init__()
        self.networks = nn.ModuleList(networks)
        self.power = nn.Parameter(torch.FloatTensor([1]))
        
    @torch.cuda.amp.autocast()
    def forward(
        self,
        user_id, content_id, content_type_id, part_id, encountered_id,
        prior_question_elapsed_time, prior_question_had_explanation,
        correctness_id, bundle_id, cont_streaks,
        continuous, session, diagnostic, padding_mask, task_container_id
    ):
        futures = [
            torch.jit.fork(
                model,
                user_id, content_id, content_type_id, part_id, encountered_id,
                prior_question_elapsed_time, prior_question_had_explanation,
                correctness_id, bundle_id, cont_streaks,
                continuous, session, diagnostic, padding_mask, task_container_id
            ) for model in self.networks
        ]
        outputs = [torch.jit.wait(fut) for fut in futures]
        return (torch.stack(outputs) ** self.power).mean(dim=0)

# Gym

In [ ]:
class LinearWarmupStepSchedule(torch.optim.lr_scheduler.LambdaLR):
    def __init__(self, optimizer, warmup_steps):
        self.warmup_steps = warmup_steps
        super(LinearWarmupStepSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=-1)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return (step+1) / (max(1., self.warmup_steps))

        return 1

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.vals = []

    def update(self, val, n=1):
        self.vals.append(val)
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
    def draw(self, epoch, score, prefix=''):
        plt.title(f'{prefix} E{epoch+1}, Loss: {self.avg:.5f}, AUC: {score:.5f}')
        plt.plot(self.vals, linewidth=0.2)
        plt.show()

In [ ]:
def draw(trn_loss, trn_auc, val_loss, val_auc):
    slope_trn, intercept_trn = scipy.stats.linregress(np.arange(len(trn_loss)), trn_loss)[:2]
    
    plt.title(f'E{epoch+1}. L: {np.mean(trn_loss):.5f}, {np.mean(val_loss):.5f}.  AUC: {trn_auc:.5f}, {val_auc:.5f}')
    plt.plot(trn_loss, linewidth=0.2, label=f'Trn {slope_trn}')
    plt.plot(val_loss, linewidth=0.2, label=f'Val')
    plt.plot([0,len(trn_loss)], [intercept_trn, intercept_trn + slope_trn*len(trn_loss)], c='blue', linewidth=1)
    plt.legend()
    plt.show()
    
    print(f'- L: {np.mean(trn_loss):.4f}, {np.mean(val_loss):.4f}')
    print(f'- A: {trn_auc:.4f}, {val_auc:.4f}')
    
    return [np.mean(trn_loss), np.mean(val_loss), trn_auc, val_auc]

In [ ]:
class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = None
        self.y_pred = None
        self.all_preds = None # Not just the masked ones
        self.all_masks = None # ..
        self.score = 0

    def update(self, y_true, y_pred, mask):
        # NOTE: 1 = CORRECT!!!!!!
        y_true = y_true.detach().cpu().numpy()
        #y_true = y_true.detach().cpu().argmax(-1).numpy()
        #y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        
        #y_pred = torch.softmax(y_pred.detach().float(), dim=2).data.cpu().numpy()[:,:,1]
        #y_pred = torch.sigmoid(y_pred.detach()).data.cpu().numpy()[:,:,1] # 0 = correct xent incorrect actuakky
        
        y_pred = torch.sigmoid(y_pred.detach()).data.cpu().numpy() # 0 = correct bce
        #y_pred = torch.softmax(y_pred.detach(), dim=2).data.cpu().numpy()[:,:,1]
        
        # IRT
        #y_pred = y_pred.detach().data.cpu().numpy() # 0 = correct

        mask = mask.detach().cpu().numpy()
        
        if self.y_true is None:
            self.y_true = y_true[~mask]
            self.y_pred = y_pred[~mask]
            #self.all_masks = mask[None]
            #self.all_preds = y_pred[None]
        else:
            self.y_true = np.concatenate((self.y_true, y_true[~mask]))
            self.y_pred = np.concatenate((self.y_pred, y_pred[~mask]))
            
            # We can add these later if we need them...
            #print(self.all_preds.shape, y_pred[None].shape)
            #self.all_preds = np.vstack((self.all_preds, y_pred[None]))
            #self.all_masks = np.vstack((self.all_masks, mask[None]))

    @property
    def avg(self):
        return roc_auc_score(self.y_true.flatten(), self.y_pred.flatten())
        #return auc(self.y_true.flatten(), self.y_pred.flatten())

In [ ]:
def save(path, model, optimizer, best_loss, epoch, step_scheduler=None, scheduler=None, lean=False):
    model.eval()
    
    if lean:
        torch.save(model.state_dict(), path)
        return

    params = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_loss': best_loss,
        'epoch': epoch,
    }
    if scheduler is not None: params['scheduler_state_dict'] = scheduler.state_dict()
    if step_scheduler is not None: params['step_scheduler_state_dict'] = step_scheduler.state_dict()
    torch.save(params, path)

def load(path, model, optimizer, step_scheduler=None, scheduler=None):
    checkpoint = torch.load(path)
    
    if 'step_scheduler' in checkpoint:
        step_scheduler.load_state_dict(checkpoint['step_scheduler_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return (
        model, optimizer,
        checkpoint['best_loss'], checkpoint['epoch'],
        step_scheduler, scheduler
    )

In [ ]:
def train_fn(model, optimizer, step_scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    scaler = torch.cuda.amp.GradScaler()
    meter = RocAucMeter()
    losses = AverageMeter()
    
    pbar = tqdm(dataloader)
    for idx, batch in enumerate(pbar):
        (
            user_id, content_id, content_type_id, part_id, encountered_id,
            prior_question_elapsed_time, prior_question_had_explanation,
            correctness_id, bundle_id, cont_streaks,
            continuous, session, diagnostic, mask, lecture_mask, labels, task_container_id
        ) = batch

        user_id = user_id.to(device)
        content_id = content_id.to(device)
        part_id = part_id.to(device)
        encountered_id = encountered_id.to(device)
        content_type_id = content_type_id.to(device)
        prior_question_elapsed_time = prior_question_elapsed_time.to(device)
        prior_question_had_explanation = prior_question_had_explanation.to(device)
        correctness_id = correctness_id.to(device)
        bundle_id = bundle_id.to(device)
        cont_streaks = cont_streaks.to(device)
        continuous = continuous.to(device)
        session = session.to(device)
        diagnostic = diagnostic.to(device)
        mask = mask.to(device)
        lecture_mask = lecture_mask.to(device)
        labels = labels.to(device)
        task_container_id = task_container_id.to(device)
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(
                user_id, content_id, content_type_id, part_id, encountered_id,
                prior_question_elapsed_time, prior_question_had_explanation,
                correctness_id, bundle_id, cont_streaks,
                continuous, session, diagnostic, mask, task_container_id
            )
            with torch.no_grad(): masked = mask | lecture_mask
            loss = loss_fn(output[~masked].view(-1), labels[~masked].view(-1))
            #
        
        scaler.scale(loss).backward() # loss.backward()
        scaler.step(optimizer)        # optimizer.step()
        scaler.update()
        if step_scheduler is not None: step_scheduler.step()
        
        with torch.no_grad():
            losses.update(loss.item())
            meter.update(labels, output, mask|lecture_mask)
            pbar.set_description(f'{loss.item():.4f}')
    
    return losses, meter

In [ ]:
def valid_fn(model, scheduler, loss_fn, dataloader, device):
    model.eval()
    
    meter = RocAucMeter()
    losses = AverageMeter()
    valid_preds = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader)
        for idx, batch in enumerate(pbar):
            (
                user_id, content_id, content_type_id, part_id, encountered_id, 
                prior_question_elapsed_time, prior_question_had_explanation,
                correctness_id, bundle_id, cont_streaks,
                continuous, session, diagnostic, mask, lecture_mask, labels, task_container_id
            ) = batch

            user_id = user_id.to(device)
            content_id = content_id.to(device)
            content_type_id = content_type_id.to(device)
            part_id = part_id.to(device)
            encountered_id = encountered_id.to(device)
            prior_question_elapsed_time = prior_question_elapsed_time.to(device)
            prior_question_had_explanation = prior_question_had_explanation.to(device)
            correctness_id = correctness_id.to(device)
            bundle_id = bundle_id.to(device)
            cont_streaks = cont_streaks.to(device)
            continuous = continuous.to(device)
            session = session.to(device)
            diagnostic = diagnostic.to(device)
            mask = mask.to(device)
            lecture_mask = lecture_mask.to(device)
            labels = labels.to(device)
            task_container_id = task_container_id.to(device)
            
            with torch.cuda.amp.autocast():
                output = model(
                    user_id, content_id, content_type_id, part_id, encountered_id,
                    prior_question_elapsed_time, prior_question_had_explanation,
                    correctness_id, bundle_id, cont_streaks,
                    continuous, session, diagnostic, mask, task_container_id
                )
                with torch.no_grad(): masked = mask | lecture_mask
                loss = loss_fn(output[~masked].view(-1), labels[~masked].view(-1))

            losses.update(loss.item())
            meter.update(labels, output, mask|lecture_mask)
            pbar.set_description(f'{loss.item():.4f}')

    if scheduler is not None: scheduler.step(losses.avg)
    
    return losses, meter

In [ ]:
# def inference_fn(model, dataloader, device):
#     model.eval()
#     preds = []
    
#     for data in dataloader:
#         inputs = data['x'].to(device)

#         with torch.no_grad():
#             outputs = model(inputs)
        
#         preds.append(outputs.sigmoid().detach().cpu().numpy())
        
#     preds = np.concatenate(preds)
#     return preds

# Build out and cache data

In [ ]:
# Prepare the dataset
PREFIX = 'cvfold4'
!mkdir -p {PREFIX}

# TODO: Red the order on all of these to ensure correct
if PRELOAD:
    (
        df_train, df_valid, Ntrn, Nval,
        train_user_id_sampling, valid_user_id_sampling,
        train_user_id_indices, valid_user_id_indices,
    ) = joblib.load('./preload.jlib')
    
    (
        embedded_user_ids_map, mask_user_ids, part_ids_map, lecture_ids_map, bundle_id_map,
        question_incorrect_ranks,
        answer_ratio1_mean, answer_ratio1_std,
        answer_ratio2_mean, answer_ratio2_std,
        correct_streak_u_mean, correct_streak_u_std,
        incorrect_streak_u_mean, incorrect_streak_u_std,
        correct_streak_alltime_u_mean, correct_streak_alltime_u_std,
        incorrect_streak_alltime_u_mean, incorrect_streak_alltime_u_std,
        diagnostic_u_recency_1_mean, diagnostic_u_recency_1_std,
        diagnostic_u_recency_2_mean, diagnostic_u_recency_2_std,
        diagnostic_u_recency_3_mean, diagnostic_u_recency_3_std,
        diagnostic_u_recency_4_mean, diagnostic_u_recency_4_std,
        diagnostic_u_recency_5_mean, diagnostic_u_recency_5_std,
        diagnostic_u_recency_6_mean, diagnostic_u_recency_6_std,
        session_content_num_u_mean, session_content_num_u_std,
        session_duration_u_mean, session_duration_u_std,
        session_correctness_u_mean, session_correctness_u_std,
        session_ans_duration_u_mean, session_ans_duration_u_std,
        lifetime_ans_duration_u_mean, lifetime_ans_duration_u_std,
        correctness_u_recency_mean, correctness_u_recency_std,
        part_correctness_u_recency_mean, part_correctness_u_recency_std,
        lag_ts_u_recency_mean, lag_ts_u_recency_std,
        last_ts_u_recency1_mean, last_ts_u_recency1_std,
        last_ts_u_recency2_mean, last_ts_u_recency2_std,
        last_ts_u_recency3_mean, last_ts_u_recency3_std,
        last_incorrect_ts_u_recency_mean, last_incorrect_ts_u_recency_std,
        last_correct_ts_u_recency_mean, last_correct_ts_u_recency_std,
    ) = joblib.load('./mappings.jlib')
    
else:
    (
        df_train, df_valid, Ntrn, Nval,
        embedded_user_ids_map, mask_user_ids, part_ids_map, lecture_ids_map, bundle_id_map,
        train_user_id_sampling, valid_user_id_sampling,
        train_user_id_indices, valid_user_id_indices,
        
        question_incorrect_ranks,
        answer_ratio1_mean, answer_ratio1_std,
        answer_ratio2_mean, answer_ratio2_std,
        correct_streak_u_mean, correct_streak_u_std,
        incorrect_streak_u_mean, incorrect_streak_u_std,
        correct_streak_alltime_u_mean, correct_streak_alltime_u_std,
        incorrect_streak_alltime_u_mean, incorrect_streak_alltime_u_std,
        diagnostic_u_recency_1_mean, diagnostic_u_recency_1_std,
        diagnostic_u_recency_2_mean, diagnostic_u_recency_2_std,
        diagnostic_u_recency_3_mean, diagnostic_u_recency_3_std,
        diagnostic_u_recency_4_mean, diagnostic_u_recency_4_std,
        diagnostic_u_recency_5_mean, diagnostic_u_recency_5_std,
        diagnostic_u_recency_6_mean, diagnostic_u_recency_6_std,
        session_content_num_u_mean, session_content_num_u_std,
        session_duration_u_mean, session_duration_u_std,
        session_correctness_u_mean, session_correctness_u_std,
        session_ans_duration_u_mean, session_ans_duration_u_std,
        lifetime_ans_duration_u_mean, lifetime_ans_duration_u_std,
        correctness_u_recency_mean, correctness_u_recency_std,
        part_correctness_u_recency_mean, part_correctness_u_recency_std,
        lag_ts_u_recency_mean, lag_ts_u_recency_std,
        last_ts_u_recency1_mean, last_ts_u_recency1_std,
        last_ts_u_recency2_mean, last_ts_u_recency2_std,
        last_ts_u_recency3_mean, last_ts_u_recency3_std,
        last_incorrect_ts_u_recency_mean, last_incorrect_ts_u_recency_std,
        last_correct_ts_u_recency_mean, last_correct_ts_u_recency_std,
    ) = prepare_training_data(fold_number=4)
    
    
    joblib.dump((
        df_train, df_valid, Ntrn, Nval,
        train_user_id_sampling, valid_user_id_sampling,
        train_user_id_indices, valid_user_id_indices,
        
    ), './preload.jlib')
    joblib.dump((
            embedded_user_ids_map, mask_user_ids, part_ids_map, lecture_ids_map, bundle_id_map,
            question_incorrect_ranks,
            answer_ratio1_mean, answer_ratio1_std,
            answer_ratio2_mean, answer_ratio2_std,
            correct_streak_u_mean, correct_streak_u_std,
            incorrect_streak_u_mean, incorrect_streak_u_std,
            correct_streak_alltime_u_mean, correct_streak_alltime_u_std,
            incorrect_streak_alltime_u_mean, incorrect_streak_alltime_u_std,
            diagnostic_u_recency_1_mean, diagnostic_u_recency_1_std,
            diagnostic_u_recency_2_mean, diagnostic_u_recency_2_std,
            diagnostic_u_recency_3_mean, diagnostic_u_recency_3_std,
            diagnostic_u_recency_4_mean, diagnostic_u_recency_4_std,
            diagnostic_u_recency_5_mean, diagnostic_u_recency_5_std,
            diagnostic_u_recency_6_mean, diagnostic_u_recency_6_std,
            session_content_num_u_mean, session_content_num_u_std,
            session_duration_u_mean, session_duration_u_std,
            session_correctness_u_mean, session_correctness_u_std,
            session_ans_duration_u_mean, session_ans_duration_u_std,
            lifetime_ans_duration_u_mean, lifetime_ans_duration_u_std,
            correctness_u_recency_mean, correctness_u_recency_std,
            part_correctness_u_recency_mean, part_correctness_u_recency_std,
            lag_ts_u_recency_mean, lag_ts_u_recency_std,
            last_ts_u_recency1_mean, last_ts_u_recency1_std,
            last_ts_u_recency2_mean, last_ts_u_recency2_std,
            last_ts_u_recency3_mean, last_ts_u_recency3_std,
            last_incorrect_ts_u_recency_mean, last_incorrect_ts_u_recency_std,
            last_correct_ts_u_recency_mean, last_correct_ts_u_recency_std,
        ),
        './mappings.jlib'
    )

# TODO: Remove this: temp measure...
#df_train.loc[df_train.lag_ts_u_recency.isna(),'lag_ts_u_recency'] = -999
#df_valid.loc[df_valid.lag_ts_u_recency.isna(),'lag_ts_u_recency'] = -999
# 80488335/? [08:34<00:00, 156373.86it/s]
# 80488335/? [07:54<00:00, 169760.91it/s]
# 80488335/? [07:43<00:00, 173648.30it/s] <-- questions only
# 80488335/? [07:54<00:00, 169693.34it/s] <-- all content_id
# 80488335/? [08:11<00:00, 163828.23it/s] <-- whatever we're doing now
df_train.head(50)

In [ ]:
# print(d[0]) # user_id 115; I encourage you to match the same with the dataframes.
# z = next(iter(loader))
# len(z)

In [ ]:
df_train.shape

In [ ]:
df_valid.shape

In [ ]:
df_train.columns

In [ ]:
# # z = df_train#.sample(5000000)
# # y = df_valid#.sample(5000000)

# for f in [
#     'lag_ts_u_recency'
# #     'session_content_num_u', 'session_duration_u',
# #     'last_ts_u_recency1','last_ts_u_recency2','last_ts_u_recency3',
# #     'last_correct_ts_u_recency','last_incorrect_ts_u_recency',
# #     'correctness_u_recency','part_correctness_u_recency',
# ]:
#     ax = plt.subplots(2,2, figsize=(14,10))[1]
#     ax[0,0].set_title(f'{f} train')
#     ax[1,0].set_title(f'{f} valid')
#     ax[0,0].hist(np.log1p(df_train[f]), 30)
#     ax[1,0].hist(np.log1p(df_valid[f]), 30)
    

#     ax[0,1].set_title(f'{f} train')
#     ax[1,1].set_title(f'{f} valid')
#     ax[0,1].hist(np.log1p(df_train[f]/1000), 30)
#     ax[1,1].hist(np.log1p(df_valid[f]/1000), 30)
#     plt.show()

# Regular Training

In [ ]:
gc.collect()

def worker_init(worker_id):
    weak_seed = torch.utils.data.get_worker_info().seed % 10240
    random.seed(weak_seed)
    np.random.seed(weak_seed)
    
# Truncate Timestamp
train_dataset = Riiid(
    npdata=df_train[dataset_cols[COL_OFFSET_CUT:]].values,
    N=Ntrn, #df_train.user_id.nunique(),
    user_id_indices=train_user_id_indices,
    user_id_sampling=train_user_id_sampling,
    mask_user_ids=mask_user_ids
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=4,
    worker_init_fn=worker_init,
    pin_memory=True,
    shuffle=True # doesn't really matter since it'll be random anywaay
)


valid_dataset = Riiid(
    npdata=df_valid[dataset_cols[COL_OFFSET_CUT:]].values,
    # multiplying by 5 just so we get some coverage....
    N=Nval*5, #df_valid.user_id.nunique(),
    user_id_indices=valid_user_id_indices,
    user_id_sampling=valid_user_id_sampling,
    mask_user_ids=mask_user_ids,
    augment=False
)

valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=4,
    worker_init_fn=worker_init,
    pin_memory=True,
    shuffle=True
)
_ = gc.collect()

print(valid_dataset[0]) # sample dataset

In [ ]:
gc.collect()

# TODO:  1e-4 - 5e-5
lr = 2.5e-4 # SAINT+ paper = 1e-3
lr = 1e-3
WEIGHT_DECAY = 0.0000001 #0.00001 # significant...

# creating the model
model = SaintTransformerModel(
    d_model=128,    # minimally
    nhead=4,        # leave it
    num_layers=2,   # maybe increase it 2-4
    dropout=0.05,   # can be increased....
    emb_dropout=0.05,
    initrange=0.02
)
# lr=2e-3
# model = SiameseNetwork([
#     SaintTransformerModel(
#         d_model=128,    # minimally
#         nhead=4,        # leave it
#         num_layers=2,   # maybe increase it 2-4
#         dropout=0.1,    # can be increased....
#         emb_dropout=0.15,
#         initrange=0.02
#     ),
#     SaintTransformerModel(
#         d_model=128,    # minimally
#         nhead=4,        # leave it
#         num_layers=2,   # maybe increase it 2-4
#         dropout=0.15,   # can be increased....
#         emb_dropout=0.1,
#         initrange=0.02
#     )
# ])

# model = SaintTransformerModel(
#     d_model=256,
#     nhead=4,
#     num_layers=4,
#     dropout=0.1,
#     emb_dropout=0.1,
#     initrange=0.02
# )

# # createing the mdoel
# model = SaintTransformerModel(
#     d_model=256,    # minimally
#     nhead=4,        # leave it
#     num_layers=2,   # maybe increase it 2-4
#     dropout=0,   # can be increased....
#     emb_dropout=0,
#     initrange=0.02
# )

# # Finetuning: Load ML-Model:
# lr = 1e-4
# # state = torch.load(f'{PREFIX}/pretrain_bundle_best-loss-leanmodel.pth')
# state = torch.load(f'45-cvfold0/best-loss-leanmodel.pth')
# for key in list(state.keys()):
#     if 'user_embeddings' in key:
#         state.pop(key)
#         continue
#     state[key[len('module.'):]] = state[key]
#     del state[key]
# print(model.load_state_dict(state, strict=False))

# # Patch the decoder to accept our user embedding, which is new...
# # NOTE: We really hsould create 2 sepaarate model classes:
# for name, param in model.named_parameters():                
#     if name.startswith('decoder'): continue
#     if 'user_embeddings' in name: continue
#     param.requires_grad = False

# decoder = torch.zeros([1,128+16], dtype=torch.float32)
# decoder.data[:,:128] = model.decoder.weight.data # x.clone().detach()
# model.decoder.weight.data = decoder.data
# model.decoder.in_features = 128+16
# # self.weight = Parameter(torch.Tensor(out_features, in_features))

model.to(DEVICE) # look into it!
model = nn.DataParallel(model)

loss_fn_tr = nn.BCEWithLogitsLoss()
loss_fn_vl = nn.BCEWithLogitsLoss()

# remove biases and layernorm from weight decay and add weight decay for regularization
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'norm']
optimizer_grouped_parameters = [
    # NOTE: LayerNorm's weights + biases are both included in the do not decay list
    {'params': filter(lambda p: p.requires_grad, [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)]), 'weight_decay': WEIGHT_DECAY},
    {'params': filter(lambda p: p.requires_grad, [p for n, p in param_optimizer if any(nd in n for nd in no_decay)]), 'weight_decay': 0.0}
]
optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=lr)


warmup_step_scheduler = None
# warmup_step_scheduler = LinearWarmupStepSchedule(
#     optimizer=optimizer,
#     warmup_steps=len(train_loader)//4 # first two epochs? //3*2 # first ~67%
# )

lr_redux_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    factor=0.5,
    patience=1,
    verbose=True, 
    threshold_mode='abs',
    min_lr=1e-7,
    eps=1e-7
)

#_ = count_parameters(model)

In [ ]:
# adjusted initrange of ts and removed decoder bias 0 fill
best_loss = 999
patience = 4
log = []
gc.collect()
torch.cuda.empty_cache()

print('Full training progress...')
for epoch in tqdm(range(N_EPOCHS)):
    trn_losses, trn_meter = train_fn(
        model,
        optimizer,
        step_scheduler=warmup_step_scheduler,
        loss_fn=loss_fn_tr,
        dataloader=train_loader,
        device=DEVICE
    )
    #trn_losses.draw(epoch, trn_meter.avg, 'Train')
    
    val_losses, val_meter = valid_fn(
        model,
        scheduler=lr_redux_scheduler,
        loss_fn=loss_fn_vl,
        dataloader=valid_loader,
        device=DEVICE
    )
    #val_losses.draw(epoch, val_meter.avg, 'Valid')
    log.append(draw(trn_losses.vals, trn_meter.avg, val_losses.vals, val_meter.avg))
    
    
    if val_losses.avg >= best_loss:
        ez_out -= 1
        print(f'Loss not better... {ez_out} more chances.')
        if ez_out <= 0:
            print('Breaking!')
            break
        
    else:
        # Save model...
        best_loss = val_losses.avg
        ez_out = patience
        
        save(
            f'{PREFIX}/epoch{epoch}-checkpoint.pth',
            model, optimizer,
            best_loss,
            epoch,
            step_scheduler=warmup_step_scheduler,
            scheduler=lr_redux_scheduler,
            lean=False
        )
        
        save(
            f'{PREFIX}/best-loss-leanmodel.pth',
            model, None,
            None,
            None,
            step_scheduler=None,
            scheduler=None,
            lean=True
        )
        
        pd.DataFrame(log, columns=['TrnLoss','ValLoss','TrnAUC','ValAUC']).to_csv(f'{PREFIX}/log.csv', index=False)

# Best Val Loss
pd.DataFrame(log, columns=['TrnLoss','ValLoss','TrnAUC','ValAUC'])#.sort_values('ValLoss')
#  A: 0.7422, 0.7829 full layered history aadded to decoder input
# now testing full layer history added to clasasificaation layer

In [ ]:
# NOTE: Lag feature is completely removed......

z=pd.DataFrame(log, columns=['TrnLoss','ValLoss','TrnAUC','ValAUC'])
str(np.round(z.ValAUC,4).tolist()).replace(', 0.', ',')

In [ ]:
7795,7844,7861,7885,7886,7899,7899,7915,7920 <-- attempt_no with part_correctness_u merged into correctness MARK_A
7776,7826,7850,7878,7890,7892,7908,7914,7912,7921,7927,7931 <--- using correct ts for last_incorrect_ts_u/last_correct_ts_u (lectures dont count) 7884 train auc
7780,7828,7851,7880,7892,7892,7912,7918,7918,7926,7931,7933,7934,7935,7939 <-- lectures count as "questions". ts and ratio features build off of all content, and lectures are counted as always answered correctly (7901)
7793,7843,7873,7887,7898,7903,7911,7911,7923,7929,7931,7940,7937,7941, <-- same as above but corrected last_ts_u_recency2/3_mean s.t. not based on current ts(!) (7910 train)
7809,7849,7873,7888,7897,7904,7908,7915,7923,7926,7932,7938,7942,7942,7946 (7919) <-- refactored dataset building, trained to early stop
7783,7826,7843,7860,7865,7864,7882,7887,7889,7880,7886,7891,7894,7892,7898 (7907) <-- abs log encoded lag_ts_u_recency
7789,7846,7872,7888,7893,7912,7904,7920,7926,7929,7936,7933,7936,7933,7941,7940,7940 (7915) <-- {0,1440} clipped log encoded lag_ts_u_recency 
7796,7849,7870,7889,7896,7909,7904,7927,7923,7932,7931,7931,7936,7939,7942,7940 (7910) <--- raw lag_ts_u_recency std scaled
7810,7848,7873,7891,7895,7910,7919,7917,7931,7927,7933,7937,7940 (7903) <-- tag_group_id
7822,7871,7897,7911,7920,7928,7941,7941,7946,7954,7952,7953,7959,7957 (7925) <-- 2 layer masked history
7833,7884,7902,7917,7927,7930,7939,7944,7944,7955,7954,7956,7962,7965,7965,7971,7971,7969 (7964)<-- 1 layer masked history w/ pos emb (start=0)
7830,7881,7911,7918,7925,7931,7939,7940,7952,7952,7958,7960,7957,7964 (7951) <-- tcid_size_id
7836,7878,7905,7909,7924,7930,7941,7943,7953,7952,7964,7966,7971,7969,7974,7977 (7962) <-- session_content_num_u & session_duration_u continuous linear embedded
7839,7884,7903,7918,7930,7934,7950,7949,7957,7957,7961,7965,7969 (7943) <-- session_correctness_u
bad <-- session_correctness_u as above, with batchnorm on linears and all linear biases corrected to =False
no change <-- 256 w 4layer
7839,7881,7907,7920,7936,7941,7949,7951,7957,7964,7958,7968,7970,7967,7968,7971,7976,7974,7977,7974,7976 (7968) <-- 2,3
7838,7881,7906,7919,7935,7940,7950,7951,7956,7963,7957,7968,7970,7967,7969,7971,7976,7974,7977,7975,7976 (7967) <-- 6,9
7838,7883,7905,7921,7933,7940,7946,7947,7957,7958,7960,7968,7972,7972,7971,7981 (7971) <-- 2,3 with 5 diagnostic features target clustered
7817,7874,7899,7905,7924,7922,7927,7933,7927,7928,7932,7936,7933,7937,7936,7933,7933,7934 <-- 256 w/o no dropout at all
7843,7890,7909,7923,7939,7948,7961,7962,7982,7981,7983,7988,7992,7988,7992,7992,7995,7994,7997 (7973) <-- titos cv fold 1 w/ TCID internal shuffle aug and 5 diagnostic features..
7833,7888,7904,7922,7936,7946,7948,7954,7959,7962,7967,7972,7980,7979,7983,7982,7987 (7977) <-- session_ans_duration_u, lifetime_ans_duration_u
7833,7889,7905,7921,7937,7945,7948,7954,7971,7972,7975,7975,7980,7980,7981,7979,7984 (7970) <-- same as above but with log(ms) instead of log(s)
7832,7889,7907,7922,7935,7948,7956,7961,7955,7973,7975,7977,7981,7982,7983,7989,7990,7992,7992,7993,7996,7995,7999,7998 (8004) <-- same as above but sampling doesnt prefer tail end 3x and inter-user tcid pair-swapping
7971,7980,7976,7977,7985,7984,7993,7992,7989,7990,7993 (7985) <-- pretraining of everything except exercise embedding and then finetuning exercise embedding
7842,7889,7914,7925,7934,7944,7953,7958,7962,7965,7965,7972,7971,7972,7975,7982,7977,7981,7987,7990,7992,7990,7996 (8006) <-- user_id embedding
7829,7881,7903,7927,7938,7946,7951,7956,7965,7968,7968,7970,7984,7987,7983,7990,7987,7993,7990,7993,7996,7998,7999,7998,7999,8001,8000,7997,8004,8007,8005 [8015],8005,7999,8001,8003 Base Model (no user_id) 
7836,7892,7916,7926,7935,7939,7952,7965,7963,7965,7974,7983,7979,7983,7984,7986,7984,7979,[7995],7993,7990,7995,7989 (7985) <-- longest same answer streak
7826,7877,7905,7918,7935,7938,7949,7950,7957,7961,7962,7970,7967,7968,7982,7986,7986,7985,7993,7991,7996,7993,7994,7999,[7998],8001,8003,8002,8000, (8006) <-- encoder FFN no attention
[redo] <-- memory added as input into decoder
7842,7893,7909,7931,7939,7945,7949,7955,7953,7963,7967,7966,7965,7973,7980,7988,7991,7985,7985,7997,7995,7990,[7998],7997,7999,8000,7998 (8016) <-- 2 streak features, but im uncertain I rebuilt the dataset =( so gonna rerun again...
7836,7886,7905,7920,7932, 7937,7947,7954,7955,7962, 7962,7961,7965,7968,7974, 7973,7975,7976,7977,7983, 7974,7975,7987,7993,7994, 7992,[7995],7996,7989,7997,7997  (8007) no streaaks of any kind but lag features added
7822,7883,7906,7917,7930, 7940,7947,7952,7952,7965, 7971,7972,7973,7979,7978, 7978,7984,7988,7979,7982, 7986,7989,7989,7989,7992,7995,7992,7999,7998,8,7998,7994<-- full history encoder stack mixes in encoder base features which is added to penultimate output

we can try same as above but using the same weights...

What were on:
7831,7880,7904,7922,7929, 7940,7953,7953,7960,7960, 7972,7974,7974,7985,7988, 7988,7992,7997,7994,7999,7999,7997,8,8002,8002,8003,8002,8004,801,8004,8006,8002,8006]'


# History

Mine E7:
- L: 0.497, 0.467
- A: 0.751, 0.758

Mine E12 w/ WD:
- L: 0.500, 0.467
- A: 0.746, 0.757

Mine E8 w/ Incorrect Rank:
- L: 0.498, 0.467
- A: 0.753, 0.759

Mine E12 w/ BXE
- L: 0.491, 0.466
- A: 0.759, 0.760

Lectures E5
- L: 0.485, 0.457
- A: 0.753, 0.761

Static Pos Emb E3
- L: 0.479, 0.455
- A: 0.763, 0.764

Sin Pos Emb E11
- L: 0.474, 0.453
- A: 0.769, 0.767

TS Features E11
- L: 0.472, 0.452
- A: 0.772, 0.769

BundleID + Linear TS Diff E9:
- L: 0.442, 0.423
- A: 0.793, 0.787

EncoderQuestion, DecoderResponse
- L: 0.4436, 0.4216
- A: 0.7903, 0.7874

Swap Memory and Output on Decoder E6 <-- good, we'll ensemble with this
- L: 0.4456, 0.4215
- A: 0.7888, 0.7878

Encoder Dropout @20% E8 <-- good, we need to increase dropout
- L: 0.4456, 0.4206
- A: 0.7888, 0.7879

PQExplanation and ContentTypeId E8 <-- not saturated due to dropout
- L: 0.4454, 0.4201
- A: 0.7890, 0.7885

Same w/ DataParallel to effectively half BS, E11
- L: 0.4425, 0.4202
- A: 0.7917, 0.7892

ItemResponseTheory instead of sigmoid E9
- L: 0.4465, 0.4204
- A: 0.7893, 0.7884

IRT w/ HeavyEncoder (all inputs except correctness+pos) E9
- L: 0.4453, 0.4197
- A: 0.7908, 0.7893

HeaviestEncoder = as above, no decoder and 4 encoder layers E7
- L: 0.4462, 0.4203
- A: 0.7892, 0.7892

8 Layer Encoder Stack E10 (HeaviestEncoder) trained with 50% LR
- L: 0.4440, 0.4189
- A: 0.7919, 0.7913

512 Dim Encoder Stack (HeaviestEncoder) E8
- L: 0.4447, 0.4199
- A: 0.7912, 0.7903 <-- standard embedded dropout = 0.2

- L: 0.4418, 0.4201
- A: 0.7948, 0.7909 <-- when we increase embedded dropout = 0.4

TS Offset (256dim 4layer-encoder only) E10
- L: 0.4425, 0.4201
- A: 0.7937, 0.7894
- this is identical to HeaviestEncoder but with the TSOffset Input. It doesn't seem to help..

Extended Train CAP_SAMPLING_LENGTH E9 (HeaviestEncoder)
- L: 0.4644, 0.4178
- A: 0.7885, 0.7914

User Correctness CumMean (Based on above model) E17
- L: 0.4574, 0.4164
- A: 0.7965, 0.7949

Tag0 - Once again, raw tags are useless
- L: 0.4600, 0.4158
- A: 0.7934, 0.7945

Tag Community Feature E18
- L: 0.4563, 0.4157
- A: 0.7978, 0.7959

Timestamp Embedding E12
- L: 0.4596, 0.4165
- A: 0.7940, 0.7945

Full Embedding FFN w/o timestamp/tag
- L: 0.4597, 0.4160
- A: 0.7943, 0.7951

384 Emb size w/ 8layer stack - current whatever E21 with lr//5
- L: 0.4571, 0.4139
- A: 0.7956, 0.7958

fixed std,mean and added adaptive chance for IPT

Regular size. With bad std/mean for user part correctness. E 10
- L: 0.4602, 0.4165
- A: 0.7932, 0.7939

Using user part correctness adjusted per question difficulty
- ...

Lag Feature
- Hurts...

No_Responses
- L: 0.4555, 0.4133
- A: 0.7975, 0.7973

No_Responses Categorical
- L: 0.4583, 0.4141
- A: 0.7944, 0.7967

timestamp_u_incorrect_recency, timestamp_u_correct_recency features
- L: 0.4563, 0.4129
- A: 0.7967, 0.7974

patched masking
- L: 0.5052, 0.5121
- A: 0.7961, 0.7977

# Fin~